# Zuora/Eloqua export/import service

Everything you need to know about migrating data from the Zuora subscription service to the Eloqua marketing service.


Zuora export service, responsible for exporting any object SQL-style call in CSV/JSON format
More on that here: https://knowledgecenter.zuora.com/DC_Developers/M_Export_ZOQL

zuora export service?


In [ ]:
var importer = require('../Core');
var xlsx = require('xlsx');
var request = importer.import('request polyfill');

function getAuthHeaders(zuoraConfig) {
    if (typeof zuoraConfig === 'undefined'
        || zuoraConfig === null
        || typeof zuoraConfig.rest_api_user === 'undefined'
        || typeof zuoraConfig.rest_api_password === 'undefined'
        || typeof zuoraConfig.rest_api_url === 'undefined') {
        return Promise.resolve([]).then(() => {
            throw new Error('Please supply valid zuora configuration.');
        });
    }
    return {
        'Content-Type': 'application/json',
        'apiAccessKeyId': zuoraConfig.rest_api_user,
        'apiSecretAccessKey': zuoraConfig.rest_api_password,
        'Accept': 'application/json'
    };
}

function createBulkExportJob(query, zuoraConfig) {
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/object/export',
        json: query,
        method: 'POST',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => r.body.Id)
}

function getBulkExportJobStatus(exportId, zuoraConfig) {
    console.log('waiting...');
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/object/export/' + exportId,
        method: 'GET',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => {
        if (r.body.Status === 'Completed') {
            return r.body.FileId;
        } else if (r.body.Status === 'Processing' || r.body.Status === 'Pending') {
            return new Promise(resolve => setTimeout(resolve, 500))
                .then(() => getBulkExportJobStatus(exportId, zuoraConfig));
        } else {
            throw new Error('Export status error ' + r.statusCode + ' ' + r.body.Status);
        }
    });
}

function getBulkExportFile(fileId, zuoraConfig) {
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/files/' + fileId,
        method: 'GET',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => r.body)
}

function csvToJson(csv) {
    const workbook = xlsx.read(new Buffer(csv), {type:"buffer"});
    return xlsx.utils.sheet_to_json(workbook.Sheets[workbook.SheetNames[0]]);
}

module.exports = {
    csvToJson,
    createBulkExportJob,
    getBulkExportFile,
    getBulkExportJobStatus,
    getAuthHeaders
}


zuora export catalog?



In [ ]:

function getCatalog(zuoraConfig, next) {
    var catalog = [];
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + (typeof next !== 'undefined' ? next.replace(/\/v1/ig, '') : '/catalog/products'),
        method: 'GET',
        headers: getAuthHeaders(zuoraConfig)
    }).then(r => {
        catalog = catalog.concat(r.body.products);
        if(r.body.nextPage) {
            return getCatalog(zuoraConfig, r.body.nextPage).then(r => catalog.concat(r));
        }
        return catalog;
    })
}
module.exports = getCatalog;


zuora export service test?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var {
    createBulkExportJob,
    getBulkExportJobStatus,
    getBulkExportFile,
    csvToJson,
} = importer.import('zuora to eloqua.ipynb[0]');
var request = importer.import('http request polyfill');

var sandbox = sinon.createSandbox();
var zuoraConfig = {
    "rest_api_user":"devteam@fakepage.com",
    "rest_api_password":"pass",
    "rest_api_url": "http://localhost:18888"
};

describe('zuora oauth', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should connect to zuora using oauth', () => {
        const dummyId = '123';
        const dummyQuery = 'SELECT * FROM Accounts';
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {Id: dummyId } }));
        
        return createBulkExportJob(dummyQuery, zuoraConfig)
            .then(result => {
                assert.equal(result, dummyId);
                assert(requestStub.calledOnce, 'request should only be called once');
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[0].json, dummyQuery);
            });
    })
    
    it('should wait for the export to complete', () => {
        const dummyId = '12345'
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {Status: 'Completed', FileId: dummyId } }));
        
        return getBulkExportJobStatus('123', zuoraConfig)
            .then(result => {
                assert.equal(result, dummyId);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should download the csv file', () => {
        const csvFile = 'some,csv,file';
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: csvFile }));
        
        return getBulkExportFile('1234', zuoraConfig)
            .then(result => {
                assert.equal(result, csvFile);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should convert CSV to JSON', () => {
        const result = csvToJson('some,csv,file\n1,2,3');
        assert.equal(result[0].file, 3);
    })
    
})



zuora renewals query?

Zuora query, an example query that retrieves the entire list of zuora subscriptions between two dates (start, end)
The resulting JSON is passed directly to the API: https://www.zuora.com/developer/api-reference/#operation/Object_POSTExport



In [ ]:
var moment = require('moment');
var chrono = require('chrono-node');
var excludedRatePlans = [
    'Act! Pro - New License',
    'Act! Pro - 30 Day Support',
    'Act! Pro - Upgrade License',
    'Act! Password Reset Charge',
    'Act! Premium Cloud - Trial',
    'Act! Pro V19 - Upgrade License',
    'Act! Pro V20 - Upgrade License',
]
var excludedProductSkus = [
    '00000006'
]
var currencies = [
    '',
    'USD',
    'AUD',
    'NZD',
]

var query = `SELECT
    Account.Id,
    Account.Name,
    Account.AccountNumber,
    Account.resellerofRecord__c,
    Account.renewalRep__c,
    Account.commisionedSalesRep__c,
    Account.CreatedDate,
    Account.Currency,
    SoldToContact.WorkEmail,
    SoldToContact.Country,
    SoldToContact.State,
    BillToContact.WorkEmail,
    RatePlan.Id,
    RatePlan.Name,
    RatePlanCharge.Id,
    RatePlanCharge.BillingPeriod,
    RatePlanCharge.Description,
    RatePlanCharge.Quantity,
    RatePlanCharge.Version,
    RatePlanCharge.CreatedDate,
    RatePlanCharge.EffectiveEndDate,
    DefaultPaymentMethod.CreditCardExpirationMonth,
    DefaultPaymentMethod.CreditCardExpirationYear,
    DefaultPaymentMethod.CreditCardMaskNumber,
    ProductRatePlanCharge.Id,
    ProductRatePlan.planType__c,
    ProductRatePlan.planSubType__c,
    ProductRatePlan.Id,
    ProductRatePlan.Name,
    Product.productType__c,
    Product.Name,
    Product.Description,
    Product.Id,
    Product.SKU,
    Subscription.Id,
    Subscription.Name,
    Subscription.Status,
    Subscription.Reseller__c,
    Subscription.SubscriptionEndDate,
    Subscription.SubscriptionStartDate,
    Subscription.TermStartDate,
    Subscription.TermEndDate,
    Subscription.AutoRenew
FROM RatePlanCharge
WHERE Subscription.Status!='Draft' AND Subscription.Status!='Cancelled' AND Subscription.Status!='Expired'
    AND Subscription.TermEndDate &gt;='{0}' AND Subscription.TermEndDate &lt;='{1}'
    AND (Account.Currency='${currencies.join("' OR Account.Currency='")}')
    AND (ProductRatePlan.Name!='${excludedRatePlans.join("' AND ProductRatePlan.Name!='")}')
    AND (Product.SKU!='${excludedProductSkus.join("' AND Product.SKU!='")}')
    AND NOT (SoldToContact.WorkEmail LIKE 'qaaw%@gmail.com')
    AND NOT (BillToContact.WorkEmail LIKE 'qaaw%@gmail.com')
    AND NOT (Account.Name LIKE '%do not use%')
`;
// AND (RatePlanCharge.EffectiveEndDate &gt;='{2}' OR RatePlanCharge.ChargeType='OneTime')
// removed this so that discounts show up on the account
// AND RatePlanCharge.BillingPeriod!='Month'

function getQuery(start, end) {
    // TODO: add option for pulling based on subscription term or based on modified fields
    return {
        Query: query.replace('{0}', moment(chrono.parseDate(start)).format('YYYY-MM-DD'))
                    .replace('{1}', moment(chrono.parseDate(end)).format('YYYY-MM-DD'))
                    .replace('{2}', moment(new Date()).format('YYYY-MM-DD')),
        Format: 'csv',
        Zip: false
    };
}
module.exports = {
    getQuery
};


test zuora renewals query?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var renewalsQuery = importer.import('zuora renewals query');

describe('zuora query', () => {
    it('should include the dates specified', () => {
        const now = new Date();
        const year = now.getMonth() < 11 ? (now.getFullYear() - 1) : now.getFullYear()
        const q = renewalsQuery.getQuery('beginning of November', 'beginning of December');
        assert(q.Query.includes(year + '-11-01'), 'should have correct dates');
    })
})


eloqua import service?

eloqua create template, this template should match the the output from the eloqua customobject/:id/fields REST call:
More on that here: https://docs.oracle.com/cloud/latest/marketingcs_gs/OMCAC/api-Application-2.0-Custom%20object%20data.html



In [ ]:
esvar importer = require('../Core');
var request = importer.import('http request polyfill');
var {
    bulkImportTemplate,
    temporaryImportTemplate
} = importer.import('eloqua create template');

function eloquaOauth(eloquaConfig) {
    if (typeof eloquaConfig === 'undefined'
        || eloquaConfig === null
        || typeof eloquaConfig.rest_api_company === 'undefined'
        || typeof eloquaConfig.rest_api_user === 'undefined'
        || typeof eloquaConfig.rest_api_url === 'undefined'
        || typeof eloquaConfig.rest_api_password === 'undefined'
        || typeof eloquaConfig.rest_client_id === 'undefined'
        || typeof eloquaConfig.rest_secret === 'undefined') {
        return Promise.resolve([]).then(() => {
            throw new Error('Please supply valid eloqua configuration.');
        });
    }
    var authBody = {
        "grant_type": "password",
        "scope": "full",
        "username": eloquaConfig.rest_api_company + '\\' + eloquaConfig.rest_api_user,
        "password": eloquaConfig.rest_api_password
    };
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.token_uri,
        method: 'POST',
        json: authBody,
        headers: {
            'Authorization': "Basic " + new Buffer(eloquaConfig.rest_client_id + ":" + eloquaConfig.rest_secret).toString("base64"),
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
    }).then(res => {
        res.body.expires = (new Date()).getTime() + parseFloat(res.body.expires_in) * 1000;
        return res.body;
    });
}

function eloquaRequestHeaders(eloquaToken) {
    return {
        'Authorization': "Bearer " + eloquaToken.access_token,
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    };
}

function eloquaBulkImportStatus(syncUri, eloquaToken, eloquaConfig) {
    console.log(syncUri);
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0' + syncUri,
        method: 'GET',
        headers: eloquaRequestHeaders(eloquaToken)
    }).then(res => {
        if (res.body.status === 'success' || res.body.status === 'warning') {
            return true;
        } else if (res.body.status === 'active' || res.body.status === 'pending') {
            return new Promise(resolve => setTimeout(resolve, 500))
                .then(() => eloquaBulkImportStatus(syncUri, eloquaToken, eloquaConfig));
        } else {
            throw new Error('Sync status error ' + res.statusCode + ' ' + JSON.stringify(res.body));
        }
    });
}

function completeBulkImportSync(importUri, eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0/syncs',
        method: 'POST',
        json: {
            syncedInstanceUri: importUri
        },
        headers: eloquaRequestHeaders(eloquaToken)
    }).then(res => {
        const syncUri = res.body.uri;
        return eloquaBulkImportStatus(syncUri, eloquaToken, eloquaConfig);
    });
}

function startBulkImportData(json, importUri, eloquaToken, eloquaConfig) {
    if(typeof importUri !== 'string') {
        throw new Error('something is seriously wrong with this import');
    }
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0' + importUri + '/data',
        method: 'POST',
        json: json,
        headers: eloquaRequestHeaders(eloquaToken)
    });
}

// https://docs.oracle.com/cloud/latest/marketingcs_gs/OMCAB/Developers/BulkAPI/Endpoints/Custom%20objects/Imports/post-customObjects-imports.htm
function createImportDefinition(customDataObjectId, eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0/customobjects/' + customDataObjectId + '/imports',
        method: 'POST',
        json: bulkImportTemplate(customDataObjectId),
        headers: eloquaRequestHeaders(eloquaToken)
    }).then(res => {
        return res.body.uri;
    });
}

function createInstanceDefinition(instanceId, executionId, eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0/contacts/imports',
        method: 'POST',
        json: temporaryImportTemplate(instanceId, executionId),
        headers: eloquaRequestHeaders(eloquaToken)
    }).then(res => {
        return res.body.uri;
    });
}

module.exports = {
    eloquaOauth,
    createImportDefinition,
    startBulkImportData,
    completeBulkImportSync,
    eloquaBulkImportStatus,
    createInstanceDefinition,
    eloquaRequestHeaders
}


test eloqua import service?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var request = importer.import('http request polyfill');
var {
    eloquaOauth,
    createImportDefinition,
    startBulkImportData,
    completeBulkImportSync
} = importer.import('eloqua import service');

var {
    getEloquaConfig,
    getOauthToken,
    getImportData
} = importer.import('eloqua import blueprints');

var eloquaConfig = getEloquaConfig();
var eloquaToken = getOauthToken();
var sandbox = sinon.createSandbox();

describe('eloqua bulk upload', () => {
        
    afterEach(() => {
        sandbox.restore();
    })

    it('should get a valid oauth token', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {expires_in: 1000 } }));
        
        return eloquaOauth(eloquaConfig)
            .then(r => {
                assert(r.expires > (new Date()).getTime());
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should create a bulk import instance', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {uri: '/imports/123' } }));
        
        return createImportDefinition(60, eloquaToken, eloquaConfig)
            .then(r => {
                assert(r.includes('/imports/'));
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should update data to eloqua', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {uri: '/imports/123' }, statusCode: 204 }));
        
        return startBulkImportData([getImportData()], '/imports/123', eloquaToken, eloquaConfig)
            .then(r => {
                assert(r.statusCode === 204, 'invalid status recieved from import ' + r.statusCode);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should verify upload was successful', () => {
        var importUri;
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {uri: '/imports/123', status: 'success' } }));
        
        return completeBulkImportSync(importUri, eloquaToken, eloquaConfig)
            .then(r => {
                assert(r === true);
                assert(requestStub.called, 'request should only be called once');
            })
    })
    
})


zuora eloqua mapper?



In [ ]:
var moment = require('moment');
var _ = require('lodash');
var chrono = require('chrono-node');

function mapRatePlanToProduct(description) {
    description = description.toLowerCase();
    if(description.includes('trial'))
        return 'trial';
    else if (description.includes('volume') || description.includes('discount'))
        return 'discount';
    else if (description.includes('cloud') && !description.includes('trial'))
        return 'actpremiumcloud';
    else if (description.includes('premium') && !description.includes('cloud')
             && !description.includes('trial') && !description.includes('support'))
        return 'actpremium';
    else if (description.includes('pro')  && !description.includes('support'))
        return 'actpro';
    else if (description.includes('support'))
        return 'support';
    else if (description.includes('handheld') || description.includes('contact'))
        return 'handheld';
    else if (description.includes('aem'))
        return 'aem';
    else if (description.includes('password'))
        return 'password';
    else if (description.includes('training'))
        return 'training';
    else throw new Error('product not recognized ' + description);
}

function mapDataToFields(records) {
    var uniqueIds = _.groupBy(records, r => r['Account.Id']);
    return Object.keys(uniqueIds).map(k => {
        const rpcs = _.groupBy(uniqueIds[k], r => r['RatePlanCharge.Id']);
        const charges = Object.keys(rpcs).map(k => _.sortBy(rpcs[k], r => r['RatePlanCharge.Version']).pop());
        const record = {};
        charges.sort((a, b) =>
                          chrono.parseDate(b['Subscription.TermEndDate']).getTime()
                          - chrono.parseDate(a['Subscription.TermEndDate']).getTime());
        // contact information
        const contact = charges.filter(charge => charge['SoldToContact.WorkEmail'] || charge['BillToContact.WorkEmail'])[0]
        if(typeof contact === 'undefined') {
            console.log('No contact information for ' + JSON.stringify(charges));
            return;
        }
        record['EmailAddress'] = contact['SoldToContact.WorkEmail'] || contact['BillToContact.WorkEmail'];
        record['State'] = contact['SoldToContact.State'];
        record['Country'] = contact['SoldToContact.Country'];
        record['Currency'] = contact['Account.Currency'];

        // primary product
        const actProduct = charges.filter(p => {
            const pname = mapRatePlanToProduct(p['ProductRatePlan.Name']);
            return pname === 'actpremiumcloud' || pname === 'actpremium' || pname === 'actpro' || pname === 'trial'
        })[0];
        
        if(typeof actProduct !== 'undefined') {
            record['ActProduct'] = mapRatePlanToProduct(actProduct['ProductRatePlan.Name']);
            record['Quantity'] = actProduct['RatePlanCharge.Quantity'];
        } else {
            record['ActProduct'] = 'Unknown';
            record['Quantity'] = 0;
        }
        
        // discounts!
        const discount = charges.filter(p => {
            const pname = mapRatePlanToProduct(p['ProductRatePlan.Name']);
            return pname === 'discount'
        })[0];
        if(typeof discount !== 'undefined') {
            record['Discount'] = discount['ProductRatePlan.Name'];
        } else {
            record['Discount'] = '';
        }
        
        // support
        const support = charges.filter(charge => {
            return mapRatePlanToProduct(charge['ProductRatePlan.Name']) === 'support'
        })[0];
        if(typeof support !== 'undefined') {
            record['Support'] = support['ProductRatePlan.Name'];
            record['SupportQuantity'] = support['RatePlanCharge.Quantity'];
        } else {
            record['Support'] = 'Unknown';
            record['SupportQuantity'] = 0;
        }
        
        // subscription
        const renewal = chrono.parseDate(charges[0]['Subscription.TermEndDate']);
        record['RenewalsStatus'] = charges[0]['Subscription.Status'];
        record['RenewalDate'] = moment(renewal).format('YYYY-MM-DD');
        
        // card expiration
        const expiration = new Date();
        expiration.setDate(1);
        expiration.setMonth(parseInt(charges[0]['DefaultPaymentMethod.CreditCardExpirationMonth']) - 1);
        expiration.setYear(parseInt(charges[0]['DefaultPaymentMethod.CreditCardExpirationYear']));
        record['CardExpiration'] = moment(expiration).format('YYYY-MM-DD');
        //record['Last4DigitsOfCard'] = ((/([0-9]+)/ig).exec(charges[0]['DefaultPaymentMethod.CreditCardMaskNumber']) || [])[1];
        
        // account data
        record['RepName'] = charges[0]['Account.renewalRep__c'];
        record['RORName'] = charges[0]['Account.resellerofRecord__c'];
        record['RORNumber'] = ((/([0-9]+)/ig).exec(charges[0]['Account.resellerofRecord__c']) || [])[1];
        record['AccountId'] = charges[0]['Account.Id'];

        return record;
    }).filter(r => typeof r !== 'undefined');
}
module.exports = {
    mapDataToFields,
    mapRatePlanToProduct
};


zuora eloqua mapper test?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var renewalsQuery = importer.import('zuora renewals query');
var { bulkImportTemplate } = importer.import('eloqua create template');
var { getUniqueRatePlans } = importer.import('zuora account blueprints');
var { mapDataToFields, mapRatePlanToProduct } = importer.import('zuora eloqua mapper');

var zuoraQuery = renewalsQuery.getQuery('beginning of November', 'beginning of December').Query;

var accounts = getUniqueRatePlans();
for(let records of accounts) {

    describe('given account with ' + records.map(r => r['ProductRatePlan.Name']), () => {
        
        describe('contact information properties mapped', () => {
            
            it('should map EmailAddress to eloqua given SoldToContact.WorkEmail, BillToContact.WorkEmail from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['SoldToContact.WorkEmail'] || records[0]['BillToContact.WorkEmail'], mapped[0]['EmailAddress']);
            })


            it('should map State to eloqua given SoldToContact.State from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['SoldToContact.State'], mapped[0]['State']);
            })


            it('should map Country to eloqua given SoldToContact.Country from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['SoldToContact.Country'], mapped[0]['Country']);
            })


            it('should map Currency to eloqua given Account.Currency from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['Account.Currency'], mapped[0]['Currency']);
            })

        });

        if(mapRatePlanToProduct(records[0]['ProductRatePlan.Name']) === 'pro'
            || mapRatePlanToProduct(records[0]['ProductRatePlan.Name']) === 'cloud'
            || mapRatePlanToProduct(records[0]['ProductRatePlan.Name']) === 'premium') {
            describe('primary product properties mapped', () => {

                it('should map ActProduct to eloqua given ProductRatePlan.Name from zuora', () => {
                    const mapped = mapDataToFields(records);
                    assert.equal(records[0]['ProductRatePlan.Name'], mapped[0]['ActProduct']);
                })


                it('should map Quantity to eloqua given RatePlanCharge.Quantity from zuora', () => {
                    const mapped = mapDataToFields(records);
                    assert.equal(records[0]['RatePlanCharge.Quantity'], mapped[0]['Quantity']);
                })

            });
        }

        if(mapRatePlanToProduct(records[0]['ProductRatePlan.Name']) === 'discount') {
            describe('discounts! properties mapped', () => {

                it('should map Discount to eloqua given ProductRatePlan.Name from zuora', () => {
                    const mapped = mapDataToFields(records);
                    assert.equal(records[0]['ProductRatePlan.Name'], mapped[0]['Discount']);
                })

            });
        }

        if(mapRatePlanToProduct(records[0]['ProductRatePlan.Name']) === 'support') {
            describe('support properties mapped', () => {

                it('should map Support to eloqua given ProductRatePlan.Name from zuora', () => {
                    const mapped = mapDataToFields(records);
                    assert.equal(records[0]['ProductRatePlan.Name'], mapped[0]['Support']);
                })


                it('should map SupportQuantity to eloqua given RatePlanCharge.Quantity from zuora', () => {
                    const mapped = mapDataToFields(records);
                    assert.equal(records[0]['RatePlanCharge.Quantity'], mapped[0]['SupportQuantity']);
                })

            });
        }

        describe('subscription properties mapped', () => {
            
            it('should map RenewalsStatus to eloqua given Subscription.Status from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['Subscription.Status'], mapped[0]['RenewalsStatus']);
            })

        });

        describe('account data properties mapped', () => {
            
            it('should map RepName to eloqua given Account.renewalRep__c from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['Account.renewalRep__c'], mapped[0]['RepName']);
            })


            it('should map RORName to eloqua given Account.resellerofRecord__c from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['Account.resellerofRecord__c'], mapped[0]['RORName']);
            })


            it('should map AccountId to eloqua given Account.Id from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['Account.Id'], mapped[0]['AccountId']);
            })

        });

    })

}

describe('check field map', () => {
    it('should map all fields from zuora query', () => {
        const requireFields = [
        "Account.Id",
        "RatePlanCharge.Id",
        "RatePlanCharge.Version",
        "Subscription.TermEndDate",
        "Subscription.TermEndDate",
        "SoldToContact.WorkEmail",
        "BillToContact.WorkEmail",
        "SoldToContact.WorkEmail",
        "BillToContact.WorkEmail",
        "SoldToContact.State",
        "SoldToContact.Country",
        "Account.Currency",
        "ProductRatePlan.Name",
        "ProductRatePlan.Name",
        "RatePlanCharge.Quantity",
        "ProductRatePlan.Name",
        "ProductRatePlan.Name",
        "ProductRatePlan.Name",
        "ProductRatePlan.Name",
        "RatePlanCharge.Quantity",
        "Subscription.TermEndDate",
        "Subscription.Status",
        "DefaultPaymentMethod.CreditCardExpirationMonth",
        "DefaultPaymentMethod.CreditCardExpirationYear",
        "Account.renewalRep__c",
        "Account.resellerofRecord__c",
        "Account.resellerofRecord__c",
        "Account.Id"
        ];
        const missing = requireFields.filter(f => !zuoraQuery.includes(f));
        assert(missing.length === 0, 'missing fields from zuora query ' + missing.join(', '));
    })

    it('should map all fields in eloqua import definition', () => {
        const mappedFields = [
        "EmailAddress",
        "State",
        "Country",
        "Currency",
        "ActProduct",
        "Quantity",
        "ActProduct",
        "Quantity",
        "Discount",
        "Discount",
        "Support",
        "SupportQuantity",
        "Support",
        "SupportQuantity",
        "RenewalsStatus",
        "RenewalDate",
        "CardExpiration",
        "RepName",
        "RORName",
        "RORNumber",
        "AccountId"
        ];
        const unmapped = Object.keys(bulkImportTemplate(60).fields).filter(k => !mappedFields.includes(k));
        assert(unmapped.length === 0, 'unmapped eloqua fields ' + unmapped.join(', '));
    })

    it('should map all record fields to eloqua fields', () => {
        const mappedFields = [
        "EmailAddress",
        "State",
        "Country",
        "Currency",
        "ActProduct",
        "Quantity",
        "ActProduct",
        "Quantity",
        "Discount",
        "Discount",
        "Support",
        "SupportQuantity",
        "Support",
        "SupportQuantity",
        "RenewalsStatus",
        "RenewalDate",
        "CardExpiration",
        "RepName",
        "RORName",
        "RORNumber",
        "AccountId"
        ];
        const missing = mappedFields.filter(f => !Object.keys(bulkImportTemplate(60).fields).includes(f));
        assert(missing.length === 0, 'missing fields from eloqua import definition ' + missing.join(', '));
    })
})

zuora account blueprints?


In [ ]:
function getUniqueRatePlans() {
    return [{
        "Account.Id": "2c92a0fb4a38064e014a69432bb9310d",
        "Account.Name": "Soundside Wealth Advisors",
        "Account.AccountNumber": "A00000593",
        "Account.CreatedDate": "12/20/14",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "jon.kagan@raymondjames.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Florida",
        "BillToContact.WorkEmail": "jon.kagan@raymondjames.com",
        "RatePlan.Id": "2c92a0ac5aff3e39015b12db28c731bf",
        "RatePlan.Name": "Act! Premium Cloud - Monthly Subscription",
        "RatePlanCharge.Id": "2c92a0ac5aff3e39015b12db28c731c0",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Entitles customer to a monthly subscription of Act! Premium Cloud.",
        "RatePlanCharge.Quantity": "2",
        "RatePlanCharge.Version": "4",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/9/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "1",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************7428",
        "ProductRatePlanCharge.Id": "2c92a0fb4cc7343c014cccf9e33b6a5f",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fb4cc7343c014cccf9e10f69f0",
        "ProductRatePlan.Name": "Act! Premium Cloud - Monthly Subscription (Volume)",
        "Product.productType__c": "ActPremiumCloud",
        "Product.Name": "Act! Premium Cloud",
        "Product.Id": "2c92a0fb4a38064e014a55db0e66551c",
        "Product.SKU": "SKU-00000002",
        "Subscription.Id": "2c92a0ac5aff3e39015b12db289431b4",
        "Subscription.Name": "A-S00001255",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/9/18",
        "Subscription.SubscriptionStartDate": "3/9/15",
        "Subscription.TermStartDate": "3/9/17",
        "Subscription.TermEndDate": "3/9/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fb4a38064e014a69432bb9310d",
        "Account.Name": "Soundside Wealth Advisors",
        "Account.AccountNumber": "A00000593",
        "Account.CreatedDate": "12/20/14",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "jon.kagan@raymondjames.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Florida",
        "BillToContact.WorkEmail": "jon.kagan@raymondjames.com",
        "RatePlan.Id": "2c92a0ac5aff3e39015b12db28d831c9",
        "RatePlan.Name": "Act! Premium Hosted",
        "RatePlanCharge.Id": "2c92a0ac5aff3e39015b12db28d831ca",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Quantity": "2",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "6/9/15",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "1",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************7428",
        "ProductRatePlanCharge.Id": "2c92a0fb4a38064e014a55e16ec37663",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fb4a38064f014a55deab0667c3",
        "ProductRatePlan.Name": "Act! Premium Hosted",
        "Product.productType__c": "ActPremiumCloud",
        "Product.Name": "Act! Premium Cloud",
        "Product.Id": "2c92a0fb4a38064e014a55db0e66551c",
        "Product.SKU": "SKU-00000002",
        "Subscription.Id": "2c92a0ac5aff3e39015b12db289431b4",
        "Subscription.Name": "A-S00001255",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/9/18",
        "Subscription.SubscriptionStartDate": "3/9/15",
        "Subscription.TermStartDate": "3/9/17",
        "Subscription.TermEndDate": "3/9/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fb4a38064e014a69432bb9310d",
        "Account.Name": "Soundside Wealth Advisors",
        "Account.AccountNumber": "A00000593",
        "Account.CreatedDate": "12/20/14",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "jon.kagan@raymondjames.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Florida",
        "BillToContact.WorkEmail": "jon.kagan@raymondjames.com",
        "RatePlan.Id": "2c92a0ac5aff3e39015b12db28ed31ce",
        "RatePlan.Name": "SLA Discount",
        "RatePlanCharge.Id": "2c92a0ac5aff3e39015b12db28ed31cf",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "5/9/15",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "1",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************7428",
        "ProductRatePlanCharge.Id": "2c92a0fb4c5481da014c6ca7d7056b11",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.Id": "2c92a0fb4c5481db014c6ca7474a79aa",
        "ProductRatePlan.Name": "SLA Discount",
        "Product.productType__c": "ActPremiumCloud",
        "Product.Name": "Act! Premium Cloud",
        "Product.Id": "2c92a0fb4a38064e014a55db0e66551c",
        "Product.SKU": "SKU-00000002",
        "Subscription.Id": "2c92a0ac5aff3e39015b12db289431b4",
        "Subscription.Name": "A-S00001255",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/9/18",
        "Subscription.SubscriptionStartDate": "3/9/15",
        "Subscription.TermStartDate": "3/9/17",
        "Subscription.TermEndDate": "3/9/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd5269269401526ae2acbc0147",
        "Account.Name": "Cash Solutions",
        "Account.AccountNumber": "A00061798",
        "Account.commisionedSalesRep__c": "tom.abel@swiftpage.com",
        "Account.CreatedDate": "1/22/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "todd@cashsolutions-us.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Texas",
        "BillToContact.WorkEmail": "todd@cashsolutions-us.com",
        "RatePlan.Id": "2c92a0fd5aff18a5015b12ed44483c3d",
        "RatePlan.Name": "Act! Premium Cloud - Monthly Subscription",
        "RatePlanCharge.Id": "2c92a0fd5aff18a5015b12ed44483c3e",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Act! Premium Cloud - Monthly Subscription",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "1/21/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "7",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2021",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************2600",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e235485365a19",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e2354772f1cd2",
        "ProductRatePlan.Name": "Act! Premium Cloud - Monthly Subscription",
        "Product.productType__c": "ActPremiumCloud",
        "Product.Name": "Act! Premium Cloud",
        "Product.Id": "2c92a0fb4a38064e014a55db0e66551c",
        "Product.SKU": "SKU-00000002",
        "Subscription.Id": "2c92a0fd5aff18a5015b12ed44183c34",
        "Subscription.Name": "A-S00021419",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "1/21/18",
        "Subscription.SubscriptionStartDate": "1/21/16",
        "Subscription.TermStartDate": "1/21/17",
        "Subscription.TermEndDate": "1/21/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd5269263b0152846e8fa10f76",
        "Account.Name": "WHITE LION TEA",
        "Account.AccountNumber": "A00062885",
        "Account.commisionedSalesRep__c": "Chester.Miles@Swiftpage.com",
        "Account.CreatedDate": "1/27/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "kpetty@whiteliontea.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Arizona",
        "BillToContact.WorkEmail": "kpetty@whiteliontea.com",
        "RatePlan.Id": "2c92a0fc5aff175f015b12edf9a165a6",
        "RatePlan.Name": "Act! Premium Volume Discounts (5-19 Users)",
        "RatePlanCharge.Id": "2c92a0fc5aff175f015b12edf9a165a7",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Act! Premium Volume Discounts (5-19 Users)",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "1/26/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "10",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************5614",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e2342589e3c1e",
        "ProductRatePlan.planType__c": "VolumeDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e30014e234256ec0647",
        "ProductRatePlan.Name": "Act! Premium Volume Discounts (5-19 Users)",
        "Product.productType__c": "ActPremiumDiscounts",
        "Product.Name": "Act! Volume Discounts",
        "Product.Id": "2c92a0f94e050e30014e234255e00641",
        "Product.SKU": "SKU-00000008",
        "Subscription.Id": "2c92a0fc5aff175f015b12edf96f658a",
        "Subscription.Name": "A-S00022247",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "1/26/18",
        "Subscription.SubscriptionStartDate": "1/26/16",
        "Subscription.TermStartDate": "1/26/17",
        "Subscription.TermEndDate": "1/26/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe526395ed015264ec60096671",
        "Account.Name": "Keystone Security",
        "Account.AccountNumber": "A00061497",
        "Account.commisionedSalesRep__c": "danielle.depasquale@swiftpage.com",
        "Account.CreatedDate": "1/21/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "jmilwid@keystonesecurity.ca",
        "SoldToContact.Country": "Canada",
        "SoldToContact.State": "Ontario",
        "BillToContact.WorkEmail": "jmilwid@keystonesecurity.ca",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b12ee24347f67",
        "RatePlan.Name": "Act! Premium - Monthly Subscription (Loyalty Pricing)",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b12ee24347f68",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Act! Premium - Monthly Subscription (Loyalty Pricing)",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "4/4/16",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "6",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************1076",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e235488625a7a",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Loyalty",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e235477e61ce0",
        "ProductRatePlan.Name": "Act! Premium - Monthly Subscription (Loyalty Pricing)",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0fe5af9a6b9015b12ee240e7f59",
        "Subscription.Name": "A-S00021210",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "1/20/18",
        "Subscription.SubscriptionStartDate": "1/20/16",
        "Subscription.TermStartDate": "1/20/17",
        "Subscription.TermEndDate": "1/20/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd52b4fb390152ce01f10215e3",
        "Account.Name": "Wells Fargo Insurance Services",
        "Account.AccountNumber": "A00066215",
        "Account.CreatedDate": "2/10/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "tbruns2012@gmail.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Texas",
        "BillToContact.WorkEmail": "tbruns2012@gmail.com",
        "RatePlan.Id": "2c92a0fe5af9a6b8015b12ee86fd0dc7",
        "RatePlan.Name": "Act! Premium - Monthly Subscription",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b8015b12ee86fd0dc8",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Act! Premium - Monthly Subscription",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "2/9/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "7",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2022",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************8551",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e235485d65a31",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e235477621cd6",
        "ProductRatePlan.Name": "Act! Premium - Monthly Subscription",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0fe5af9a6b8015b12ee86de0dc3",
        "Subscription.Name": "A-S00024431",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "2/9/18",
        "Subscription.SubscriptionStartDate": "2/9/16",
        "Subscription.TermStartDate": "2/9/17",
        "Subscription.TermEndDate": "2/9/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd5269263b01527a268e566315",
        "Account.Name": "Villareal & Associates",
        "Account.AccountNumber": "A00062175",
        "Account.CreatedDate": "1/25/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "vanessa@recoveryview.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "vanessa@recoveryview.com",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b12ef96be0767",
        "RatePlan.Name": "Act! Premier Support - Monthly",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b12ef96be0768",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Monthly Term of Premier Support gives you priority access to technical support, extended support hours, call-back scheduling and appointment setting so you can get answers on your schedule.",
        "RatePlanCharge.Quantity": "2",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "1/24/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "1",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********1048",
        "ProductRatePlanCharge.Id": "2c92a0fb4cc7343f014cccf565640bcc",
        "ProductRatePlan.planType__c": "Support",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fb4cc7343c014cccf561af4e12",
        "ProductRatePlan.Name": "Act! Premier Support - Monthly",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0fe5af9a6b9015b12ef96810750",
        "Subscription.Name": "A-S00021701",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "1/24/18",
        "Subscription.SubscriptionStartDate": "1/24/16",
        "Subscription.TermStartDate": "1/24/17",
        "Subscription.TermEndDate": "1/24/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd5254277c015260ce686b50db",
        "Account.Name": "Goldstar",
        "Account.AccountNumber": "A00061323",
        "Account.commisionedSalesRep__c": "Ryan.Becerra@swiftpage.com",
        "Account.CreatedDate": "1/20/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "sam.d327@gmail.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Michigan",
        "BillToContact.WorkEmail": "sam.d327@gmail.com",
        "RatePlan.Id": "2c92a0ff5af9b657015b12ef99295c38",
        "RatePlan.Name": "AEM Pro",
        "RatePlanCharge.Id": "2c92a0ff5af9b657015b12ef99295c39",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "AEM User Charges",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "1/19/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "11",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2018",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************3239",
        "ProductRatePlanCharge.Id": "2c92a0ff561458f8015629ca75ba6603",
        "ProductRatePlan.planType__c": "AemTier",
        "ProductRatePlan.planSubType__c": "Pro",
        "ProductRatePlan.Id": "2c92a0ff561458f8015629c9c4cf6394",
        "ProductRatePlan.Name": "AEM Pro",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0ff5af9b657015b12ef99115c31",
        "Subscription.Name": "A-S00021098",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "1/19/18",
        "Subscription.SubscriptionStartDate": "1/19/16",
        "Subscription.TermStartDate": "1/19/17",
        "Subscription.TermEndDate": "1/19/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff528dec7b0152a36e713e3158",
        "Account.Name": "Host.net",
        "Account.AccountNumber": "A00064320",
        "Account.renewalRep__c": "chris.large@swiftpage.com",
        "Account.commisionedSalesRep__c": "chris.large@swiftpage.com",
        "Account.CreatedDate": "2/2/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "kknauer@host.net",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Florida",
        "BillToContact.WorkEmail": "kknauer@host.net",
        "RatePlan.Id": "2c92a0ff5af9b657015b12eff5461af1",
        "RatePlan.Name": "AEM Basic",
        "RatePlanCharge.Id": "2c92a0ff5af9b657015b12eff5461af2",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "AEM User Charges",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "2/1/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "2",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********1134",
        "ProductRatePlanCharge.Id": "2c92a0fd5614305c015629cbaa7d1dc9",
        "ProductRatePlan.planType__c": "AemTier",
        "ProductRatePlan.planSubType__c": "Basic",
        "ProductRatePlan.Id": "2c92a0fd5614305c015629caef811888",
        "ProductRatePlan.Name": "AEM Basic",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0ff5af9b657015b12eff5241aec",
        "Subscription.Name": "A-S00023228",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "2/1/18",
        "Subscription.SubscriptionStartDate": "2/1/16",
        "Subscription.TermStartDate": "2/1/17",
        "Subscription.TermEndDate": "2/1/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff528dec7b0152a36e713e3158",
        "Account.Name": "Host.net",
        "Account.AccountNumber": "A00064320",
        "Account.renewalRep__c": "chris.large@swiftpage.com",
        "Account.commisionedSalesRep__c": "chris.large@swiftpage.com",
        "Account.CreatedDate": "2/2/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "kknauer@host.net",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Florida",
        "BillToContact.WorkEmail": "kknauer@host.net",
        "RatePlan.Id": "2c92a0ff5af9b657015b12eff5691af8",
        "RatePlan.Name": "Act! Premium - Annual Subscription",
        "RatePlanCharge.Id": "2c92a0ff5af9b657015b12eff5691af9",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Act! Premium - Annual Subscription",
        "RatePlanCharge.Quantity": "10",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "10/30/16",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "2",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********1134",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e235486a95a49",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e235477931cda",
        "ProductRatePlan.Name": "Act! Premium - Annual Subscription",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0ff5af9b657015b12eff5241aec",
        "Subscription.Name": "A-S00023228",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "2/1/18",
        "Subscription.SubscriptionStartDate": "2/1/16",
        "Subscription.TermStartDate": "2/1/17",
        "Subscription.TermEndDate": "2/1/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff528dec7b0152a36e713e3158",
        "Account.Name": "Host.net",
        "Account.AccountNumber": "A00064320",
        "Account.renewalRep__c": "chris.large@swiftpage.com",
        "Account.commisionedSalesRep__c": "chris.large@swiftpage.com",
        "Account.CreatedDate": "2/2/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "kknauer@host.net",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Florida",
        "BillToContact.WorkEmail": "kknauer@host.net",
        "RatePlan.Id": "2c92a0ff5af9b657015b12eff5851afe",
        "RatePlan.Name": "Act! Premium Perpetual License",
        "RatePlanCharge.Id": "2c92a0ff5af9b657015b12eff5851aff",
        "RatePlanCharge.Description": "Act! Premium Perpetual License",
        "RatePlanCharge.Quantity": "10",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "2/2/16",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "2",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********1134",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e235487b65a68",
        "ProductRatePlan.planType__c": "PerpetualLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e235477cb1cde",
        "ProductRatePlan.Name": "Act! Premium Perpetual License",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0ff5af9b657015b12eff5241aec",
        "Subscription.Name": "A-S00023228",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "2/1/18",
        "Subscription.SubscriptionStartDate": "2/1/16",
        "Subscription.TermStartDate": "2/1/17",
        "Subscription.TermEndDate": "2/1/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd533b6fb701533d61cbf3522c",
        "Account.Name": "Go Convergence",
        "Account.AccountNumber": "A00073718",
        "Account.renewalRep__c": "Jacob.garcia@swiftpage.com",
        "Account.commisionedSalesRep__c": "Jacob.garcia@swiftpage.com",
        "Account.CreatedDate": "3/3/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "brian@thegoco.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Florida",
        "BillToContact.WorkEmail": "brian@thegoco.com",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b12f0f1da13fa",
        "RatePlan.Name": "Act! Premium Cloud - Annual Subscription",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b12f0f1da13fb",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Act! Premium Cloud - Annual Subscription",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/2/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "3",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********1018",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e235485825a25",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e235477471cd4",
        "ProductRatePlan.Name": "Act! Premium Cloud - Annual Subscription",
        "Product.productType__c": "ActPremiumCloud",
        "Product.Name": "Act! Premium Cloud",
        "Product.Id": "2c92a0fb4a38064e014a55db0e66551c",
        "Product.SKU": "SKU-00000002",
        "Subscription.Id": "2c92a0fe5af9a6b9015b12f0f1c113f6",
        "Subscription.Name": "A-S00027329",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/2/18",
        "Subscription.SubscriptionStartDate": "3/2/16",
        "Subscription.TermStartDate": "3/2/17",
        "Subscription.TermEndDate": "3/2/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe5316fd1d01532db19d074c78",
        "Account.Name": "Kathryn Hurley",
        "Account.AccountNumber": "A00072263",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "2/29/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "flyfisher8008@yahoo.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Washington",
        "BillToContact.WorkEmail": "flyfisher8008@yahoo.com",
        "RatePlan.Id": "2c92a0fe5af9a6b8015b12f22a833b3b",
        "RatePlan.Name": "Act! Premium - Annual Subscription (Loyalty Pricing)",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b8015b12f22a833b3c",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Act! Premium - Annual Subscription (Loyalty Pricing)",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "2/28/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "5",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************5221",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e235486395a3d",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Loyalty",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e2354777b1cd8",
        "ProductRatePlan.Name": "Act! Premium - Annual Subscription (Loyalty Pricing)",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0fe5af9a6b8015b12f22a6b3b37",
        "Subscription.Name": "A-S00026864",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "2/28/18",
        "Subscription.SubscriptionStartDate": "2/28/16",
        "Subscription.TermStartDate": "2/28/17",
        "Subscription.TermEndDate": "2/28/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fc5360a65101536285ea11624c",
        "Account.Name": "Matrix Pest Elimination, Inc.",
        "Account.AccountNumber": "A00076391",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "3/10/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "debrahinson@matrixpe.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "debrahinson@matrixpe.com",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b12f280962b05",
        "RatePlan.Name": "Act! Premium Perpetual License - Business Care Migration",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b12f280962b06",
        "RatePlanCharge.Description": "This item entitles customers to the current version when they cancel. Applies to active Business Care Customers only. This item cannot be purchased by itself.",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/10/16",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "6",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2018",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************4199",
        "ProductRatePlanCharge.Id": "2c92a0fb4cc7343f014cccf565ad0bdf",
        "ProductRatePlan.planType__c": "PerpetualLicense",
        "ProductRatePlan.planSubType__c": "BusinessCareMigration",
        "ProductRatePlan.Id": "2c92a0fb4cc7343c014cccf561bd4e14",
        "ProductRatePlan.Name": "Act! Premium Perpetual License - Business Care Migration",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0fe5af9a6b9015b12f2805f2afc",
        "Subscription.Name": "A-S00028124",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/9/18",
        "Subscription.SubscriptionStartDate": "3/9/16",
        "Subscription.TermStartDate": "3/9/17",
        "Subscription.TermEndDate": "3/9/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fc5360a65101536285ea11624c",
        "Account.Name": "Matrix Pest Elimination, Inc.",
        "Account.AccountNumber": "A00076391",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "3/10/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "debrahinson@matrixpe.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "debrahinson@matrixpe.com",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b12f280b02b0e",
        "RatePlan.Name": "Act! Premium - Annual Subscription (Business Care Bronze Additional User)",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b12f280b02b0f",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Act! Premium - Annual Subscription (Business Care Bronze Additional User)",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/9/17",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "6",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2018",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************4199",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e235488ff5a8b",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "NoSupport",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e235478001ce2",
        "ProductRatePlan.Name": "Act! Premium - Annual Subscription (Business Care Bronze Additional User)",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0fe5af9a6b9015b12f2805f2afc",
        "Subscription.Name": "A-S00028124",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/9/18",
        "Subscription.SubscriptionStartDate": "3/9/16",
        "Subscription.TermStartDate": "3/9/17",
        "Subscription.TermEndDate": "3/9/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fc5360a65101536285ea11624c",
        "Account.Name": "Matrix Pest Elimination, Inc.",
        "Account.AccountNumber": "A00076391",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "3/10/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "debrahinson@matrixpe.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "debrahinson@matrixpe.com",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b12f280cf2b17",
        "RatePlan.Name": "Act! Premium - Annual Subscription (Business Care Bronze Migration Users)",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b12f280cf2b18",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Act! Premium - Annual Subscription (Business Care Bronze Migration User)",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/9/17",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "6",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2018",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************4199",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e235489ad5a9f",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "NoSupportMigration",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e2354781d1ce4",
        "ProductRatePlan.Name": "Act! Premium - Annual Subscription (Business Care Bronze Migration Users)",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0fe5af9a6b9015b12f2805f2afc",
        "Subscription.Name": "A-S00028124",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/9/18",
        "Subscription.SubscriptionStartDate": "3/9/16",
        "Subscription.TermStartDate": "3/9/17",
        "Subscription.TermEndDate": "3/9/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd5316e8b401532efe74f24cca",
        "Account.Name": "J & B Sales Co.",
        "Account.AccountNumber": "A00072432",
        "Account.resellerofRecord__c": "4000918974 Training Solutions, Inc.",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "2/29/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "jhunt@j-bsalesco.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Arizona",
        "BillToContact.WorkEmail": "jhunt@j-bsalesco.com",
        "RatePlan.Id": "2c92a0ff5af9b657015b12f2d2ec3a18",
        "RatePlan.Name": "Additional Training Block â 2 hours",
        "RatePlanCharge.Id": "2c92a0ff5af9b657015b12f2d2ec3a19",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "2/29/16",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "9",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********3004",
        "ProductRatePlanCharge.Id": "2c92a0fa5100695b015122a032924acf",
        "ProductRatePlan.Id": "2c92a0fa5100695b015122a032714acd",
        "ProductRatePlan.Name": "Additional Training Block â 2 hours",
        "Product.productType__c": "ActPremiumCloud",
        "Product.Name": "Act! Premium Cloud",
        "Product.Id": "2c92a0fb4a38064e014a55db0e66551c",
        "Product.SKU": "SKU-00000002",
        "Subscription.Id": "2c92a0ff5af9b657015b12f2d2cf3a05",
        "Subscription.Name": "A-S00026957",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "2/28/18",
        "Subscription.SubscriptionStartDate": "2/28/16",
        "Subscription.TermStartDate": "2/28/17",
        "Subscription.TermEndDate": "2/28/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd5316e8b401532efe74f24cca",
        "Account.Name": "J & B Sales Co.",
        "Account.AccountNumber": "A00072432",
        "Account.resellerofRecord__c": "4000918974 Training Solutions, Inc.",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "2/29/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "jhunt@j-bsalesco.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Arizona",
        "BillToContact.WorkEmail": "jhunt@j-bsalesco.com",
        "RatePlan.Id": "2c92a0ff5af9b657015b12f2d3093a2f",
        "RatePlan.Name": "QuickStart Training Package",
        "RatePlanCharge.Id": "2c92a0ff5af9b657015b12f2d3093a30",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "2/29/16",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "9",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********3004",
        "ProductRatePlanCharge.Id": "2c92a0f9501f225301503e29a9aa2d40",
        "ProductRatePlan.planType__c": "LaunchSupport",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fb501f2f5701503e28e37318fe",
        "ProductRatePlan.Name": "QuickStart Training Package",
        "Product.productType__c": "ActPremiumCloud",
        "Product.Name": "Act! Premium Cloud",
        "Product.Id": "2c92a0fb4a38064e014a55db0e66551c",
        "Product.SKU": "SKU-00000002",
        "Subscription.Id": "2c92a0ff5af9b657015b12f2d2cf3a05",
        "Subscription.Name": "A-S00026957",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "2/28/18",
        "Subscription.SubscriptionStartDate": "2/28/16",
        "Subscription.TermStartDate": "2/28/17",
        "Subscription.TermEndDate": "2/28/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd5316e8b401532efe74f24cca",
        "Account.Name": "J & B Sales Co.",
        "Account.AccountNumber": "A00072432",
        "Account.resellerofRecord__c": "4000918974 Training Solutions, Inc.",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "2/29/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "jhunt@j-bsalesco.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Arizona",
        "BillToContact.WorkEmail": "jhunt@j-bsalesco.com",
        "RatePlan.Id": "2c92a0ff5af9b657015b12f2d33c3a5e",
        "RatePlan.Name": "Act! Premium Volume Discounts (20-49 Users)",
        "RatePlanCharge.Id": "2c92a0ff5af9b657015b12f2d33c3a5f",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Act! Premium Volume Discounts (20-49 Users)",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "2/28/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "9",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********3004",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e234257fe3bf8",
        "ProductRatePlan.planType__c": "VolumeDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e30014e234256b90643",
        "ProductRatePlan.Name": "Act! Premium Volume Discounts (20-49 Users)",
        "Product.productType__c": "ActPremiumDiscounts",
        "Product.Name": "Act! Volume Discounts",
        "Product.Id": "2c92a0f94e050e30014e234255e00641",
        "Product.SKU": "SKU-00000008",
        "Subscription.Id": "2c92a0ff5af9b657015b12f2d2cf3a05",
        "Subscription.Name": "A-S00026957",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "2/28/18",
        "Subscription.SubscriptionStartDate": "2/28/16",
        "Subscription.TermStartDate": "2/28/17",
        "Subscription.TermEndDate": "2/28/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd536886600153802a8d4b0806",
        "Account.Name": "COUNTY WIDE EXTINGUISHER",
        "Account.AccountNumber": "A00078940",
        "Account.commisionedSalesRep__c": "Chester.Miles@Swiftpage.com",
        "Account.CreatedDate": "3/16/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "nick@countywideextinguisher.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Wisconsin",
        "BillToContact.WorkEmail": "nick@countywideextinguisher.com",
        "RatePlan.Id": "2c92a0fe5af9a6b8015b12f3066b4152",
        "RatePlan.Name": "Act! Pro Volume Discounts (5-9)",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b8015b12f3066b4153",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Act! Pro Volume Discounts (5-9)",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/15/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "4",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2018",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************9078",
        "ProductRatePlanCharge.Id": "2c92a0fd5316e85401531bb95b50442c",
        "ProductRatePlan.planType__c": "VolumeDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fc5316e84801531baf1f7204b0",
        "ProductRatePlan.Name": "Act! Pro Volume Discounts (5-19)",
        "Product.productType__c": "ActProDiscounts",
        "Product.Name": "Act! Pro Volume Discounts",
        "Product.Id": "2c92a0fd5316e85401531bab07f12625",
        "Product.SKU": "SKU-00000017",
        "Subscription.Id": "2c92a0fe5af9a6b8015b12f306384149",
        "Subscription.Name": "A-S00028749",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/15/18",
        "Subscription.SubscriptionStartDate": "3/15/16",
        "Subscription.TermStartDate": "3/15/17",
        "Subscription.TermEndDate": "3/15/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff535fffdb0153812b8c2f09d3",
        "Account.Name": "J and S Valve",
        "Account.AccountNumber": "A00079132",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "3/16/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "smb@jandsvalve.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Texas",
        "BillToContact.WorkEmail": "smb@jandsvalve.com",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b12f3fde6395c",
        "RatePlan.Name": "Act! Premier Support - Annual",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b12f3fde6395d",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Annual Term of Premier Support gives you priority access to technical support, extended support hours, call-back scheduling and appointment setting so you can get answers on your schedule.",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/15/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "1",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************4228",
        "ProductRatePlanCharge.Id": "2c92a0fb4cc7343f014cccf567770c97",
        "ProductRatePlan.planType__c": "Support",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fb4cc7343c014cccf561f94e1c",
        "ProductRatePlan.Name": "Act! Premier Support - Annual",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0fe5af9a6b9015b12f3fdb73953",
        "Subscription.Name": "A-S00028814",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/15/18",
        "Subscription.SubscriptionStartDate": "3/15/16",
        "Subscription.TermStartDate": "3/15/17",
        "Subscription.TermEndDate": "3/15/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd5382c3b70153a04b34e70d0c",
        "Account.Name": "NIGEL B DESIGN, INC",
        "Account.AccountNumber": "A00083154",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "natalie.sova@swiftpage.com",
        "Account.CreatedDate": "3/22/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "nigel@nigelbdesign.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "nigel@nigelbdesign.com",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b12f452083d08",
        "RatePlan.Name": "Act! Pro - Annual Support",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b12f452083d09",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Act! Pro - Annual Support",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/21/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "10",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2021",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************9119",
        "ProductRatePlanCharge.Id": "2c92a0fd55a0dc780155e1b376ef40b3",
        "ProductRatePlan.planType__c": "Support",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fd55a0dc780155e1b2c0f13c0d",
        "ProductRatePlan.Name": "Act! Pro - Annual Support",
        "Product.productType__c": "ActPro",
        "Product.Name": "Act! Pro",
        "Product.Id": "2c92a0fd5316e8b401531b4d72d6533c",
        "Product.SKU": "SKU-00000014",
        "Subscription.Id": "2c92a0fe5af9a6b9015b12f451ed3d04",
        "Subscription.Name": "A-S00029601",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/21/18",
        "Subscription.SubscriptionStartDate": "3/21/16",
        "Subscription.TermStartDate": "3/21/17",
        "Subscription.TermEndDate": "3/21/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe59b55c3c0159c2bb57cf6098",
        "Account.Name": "IT eSources",
        "Account.AccountNumber": "A00178698",
        "Account.CreatedDate": "1/21/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "gioman62@gmail.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Ohio",
        "BillToContact.WorkEmail": "gioman62@gmail.com",
        "RatePlan.Id": "2c92a0fc5aff175f015b1343479f0a9a",
        "RatePlan.Name": "Act! Pro V19 - New License",
        "RatePlanCharge.Id": "2c92a0fc5aff175f015b1343479f0a9b",
        "RatePlanCharge.Description": "Act! Pro V19 - New License",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "1/21/17",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "3",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************6282",
        "ProductRatePlanCharge.Id": "2c92a0fd596d31c901598ac219704f2a",
        "ProductRatePlan.planType__c": "PerpetualLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fd596d31c901598ac13fa64c78",
        "ProductRatePlan.Name": "Act! Pro V19 - New License",
        "Product.productType__c": "ActProV19",
        "Product.Name": "Act! Pro V19",
        "Product.Id": "2c92a0ff596d3b4801598abae5c8030f",
        "Product.SKU": "SKU-00000023",
        "Subscription.Id": "2c92a0fc5aff175f015b134347880a96",
        "Subscription.Name": "A-S00073405",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "1/20/18",
        "Subscription.SubscriptionStartDate": "1/20/17",
        "Subscription.TermStartDate": "1/20/17",
        "Subscription.TermEndDate": "1/20/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd536886bd01537c380d62270b",
        "Account.Name": "Peace and Grace Insurance Services",
        "Account.AccountNumber": "A00078521",
        "Account.resellerofRecord__c": "4000919068 Tech Commandos",
        "Account.renewalRep__c": "jordan.colson@swiftpage.com",
        "Account.commisionedSalesRep__c": "jordan.colson@swiftpage.com",
        "Account.CreatedDate": "3/15/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "software@techcommandos.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "gdavis@techcommandos.com",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b137b94a87e98",
        "RatePlan.Name": "Platinum Discount Annual",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b137b94a87e99",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "This discount is available to Act! Consultants at the Platinum tier. Applying this discount will lower the subscription price but will prevent them from receiving commission. ",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/14/17",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014cccffa0f435e8",
        "ProductRatePlan.planType__c": "MarginDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94cc721de014cccff9dd17253",
        "ProductRatePlan.Name": "Platinum Discount Annual",
        "Product.productType__c": "ActMarginDiscounts",
        "Product.Name": "Act! Margin",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c823cdc",
        "Product.SKU": "SKU-00000006",
        "Subscription.Id": "2c92a0fe5af9a6b9015b137b94787e8f",
        "Subscription.Name": "A-S00028630",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/14/18",
        "Subscription.SubscriptionStartDate": "3/14/16",
        "Subscription.TermStartDate": "3/14/17",
        "Subscription.TermEndDate": "3/14/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff5316fcff01532ef897597de9",
        "Account.Name": "H2O Systems, Inc.",
        "Account.AccountNumber": "A00072428",
        "Account.resellerofRecord__c": "4001372618 Keystroke Quality Computing Inc.",
        "Account.renewalRep__c": "laczy.frazee@swiftpage.com",
        "Account.commisionedSalesRep__c": "laczy.frazee@swiftpage.com",
        "Account.CreatedDate": "2/29/16",
        "Account.Currency": "USD",
        "SoldToContact.Country": "Canada",
        "SoldToContact.State": "Alberta",
        "BillToContact.WorkEmail": "ken@kqc.ca",
        "RatePlan.Id": "2c92a0ff5af9b659015b137ba18b3fae",
        "RatePlan.Name": "Partner KQC Margin Discount",
        "RatePlanCharge.Id": "2c92a0ff5af9b659015b137ba18b3faf",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "This discount is available to Act! Consultants at the KQC tier. Applying this discount will lower the subscription price but will prevent them from also receiving commission.",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "1/30/17",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c83014e234f439a6797",
        "ProductRatePlan.planType__c": "MarginDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e234f416b15b8",
        "ProductRatePlan.Name": "Canadian Var Margin Discount",
        "Product.productType__c": "ActMarginDiscounts",
        "Product.Name": "Act! Margin",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c823cdc",
        "Product.SKU": "SKU-00000006",
        "Subscription.Id": "2c92a0ff5af9b659015b137ba1593fa5",
        "Subscription.Name": "A-S00026953",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "2/28/18",
        "Subscription.SubscriptionStartDate": "2/28/16",
        "Subscription.TermStartDate": "2/28/17",
        "Subscription.TermEndDate": "2/28/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe5382cf910153862824fa4835",
        "Account.Name": "American Bank & Trust",
        "Account.AccountNumber": "A00079701",
        "Account.resellerofRecord__c": "4000919116 Marketing Technology Services, Inc.",
        "Account.renewalRep__c": "chris.clary@swiftpage.com",
        "Account.commisionedSalesRep__c": "chris.clary@swiftpage.com",
        "Account.CreatedDate": "3/17/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "pbrogan@marketingtecservices.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "South Dakota",
        "BillToContact.WorkEmail": "pbrogan@marketingtecservices.com",
        "RatePlan.Id": "2c92a0ff5af9b657015b137ba6947e53",
        "RatePlan.Name": "Diamond Discount Annual",
        "RatePlanCharge.Id": "2c92a0ff5af9b657015b137ba6947e54",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "This discount is available to Act! Consultants at the Diamond tier. Applying this discount will lower the subscription price but will prevent them from receiving commission. ",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/16/17",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014cccffa09435dc",
        "ProductRatePlan.planType__c": "MarginDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94cc721de014cccff9dbf7250",
        "ProductRatePlan.Name": "Diamond Discount Annual",
        "Product.productType__c": "ActMarginDiscounts",
        "Product.Name": "Act! Margin",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c823cdc",
        "Product.SKU": "SKU-00000006",
        "Subscription.Id": "2c92a0ff5af9b657015b137ba6587e40",
        "Subscription.Name": "A-S00028982",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/16/18",
        "Subscription.SubscriptionStartDate": "3/16/16",
        "Subscription.TermStartDate": "3/16/17",
        "Subscription.TermEndDate": "3/16/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fb4d665b7f014d7c6cb28b4d8c",
        "Account.Name": "Howard Marten Technologies",
        "Account.AccountNumber": "A00011651",
        "Account.resellerofRecord__c": "4001372618 Keystroke Quality Computers",
        "Account.renewalRep__c": "laczy.frazee@swiftpage.com",
        "Account.commisionedSalesRep__c": "laczy.frazee@swiftpage.com",
        "Account.CreatedDate": "5/22/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "phil.green@howardmarten.com",
        "SoldToContact.Country": "Canada",
        "SoldToContact.State": "Ontario",
        "BillToContact.WorkEmail": "KenQuigley@kqc.ca",
        "RatePlan.Id": "2c92a0ff5af9b659015b137bbae940b5",
        "RatePlan.Name": "Act! Premium Volume Discounts (50-99 Users)",
        "RatePlanCharge.Id": "2c92a0ff5af9b659015b137bbae940b6",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Act! Premium Volume Discounts (50-99 Users)",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/25/18",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c7f014e2342584f3c0e",
        "ProductRatePlan.planType__c": "VolumeDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e30014e234256d70645",
        "ProductRatePlan.Name": "Act! Premium Volume Discounts (50-99 Users)",
        "Product.productType__c": "ActPremiumDiscounts",
        "Product.Name": "Act! Volume Discounts",
        "Product.Id": "2c92a0f94e050e30014e234255e00641",
        "Product.SKU": "SKU-00000008",
        "Subscription.Id": "2c92a0ff5af9b659015b137bbaa240a2",
        "Subscription.Name": "A-S00026680",
        "Subscription.Status": "Active",
        "Subscription.Reseller__c": "4001372618 Keystroke Quality Computing ",
        "Subscription.SubscriptionEndDate": "3/25/18",
        "Subscription.SubscriptionStartDate": "3/25/16",
        "Subscription.TermStartDate": "3/25/17",
        "Subscription.TermEndDate": "3/25/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff538652a10153a02326162c23",
        "Account.Name": "Pilot Hill Advisors",
        "Account.AccountNumber": "A00083124",
        "Account.resellerofRecord__c": "4000919068 Tech Commandos",
        "Account.renewalRep__c": "jordan.colson@swiftpage.com",
        "Account.commisionedSalesRep__c": "jordan.colson@swiftpage.com",
        "Account.CreatedDate": "3/22/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "software@techcommandos.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "New Jersey",
        "BillToContact.WorkEmail": "gdavis@techcommandos.com",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b13857ec13f5a",
        "RatePlan.Name": "Act! Premium Perpetual License  (Loyalty Pricing)",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b13857ec13f5b",
        "RatePlanCharge.Description": "Act! Premium Perpetual License (Loyalty Pricing)",
        "RatePlanCharge.Quantity": "2",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "3/23/16",
        "ProductRatePlanCharge.Id": "2c92a0fb4e86a5ff014e9989595a4872",
        "ProductRatePlan.planType__c": "PerpetualLicense",
        "ProductRatePlan.planSubType__c": "Loyalty",
        "ProductRatePlan.Id": "2c92a0fb4e86a5ff014e998959244870",
        "ProductRatePlan.Name": "Act! Premium Perpetual License  (Loyalty Pricing)",
        "Product.productType__c": "PartnerActPremium",
        "Product.Name": "Act! Partner Premium",
        "Product.Id": "2c92a0fb4e86a5ff014e998957574841",
        "Product.SKU": "SKU-00000009",
        "Subscription.Id": "2c92a0fe5af9a6b9015b13857e7d3f3f",
        "Subscription.Name": "A-S00029764",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/22/18",
        "Subscription.SubscriptionStartDate": "3/22/16",
        "Subscription.TermStartDate": "3/22/17",
        "Subscription.TermEndDate": "3/22/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fb4edd70c9014ee65661bb7804",
        "Account.Name": "Windsor Corporate Suites",
        "Account.AccountNumber": "A00025449",
        "Account.resellerofRecord__c": "4000919033 AspenTech Consulting Group, Inc.",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "7/31/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "AGeaudreau@windsorcommunities.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Massachusetts",
        "BillToContact.WorkEmail": "mknapp@aspen-tech.com",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b13985d3539ea",
        "RatePlan.Name": "2 Year Act! Premium - Annual Subscription (2 Year Contract)",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b13985d3539eb",
        "RatePlanCharge.BillingPeriod": "Two Years",
        "RatePlanCharge.Description": "2 Year Act! Premium - Annual Subscription (2 Year Contract)",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "7/30/18",
        "ProductRatePlanCharge.Id": "2c92a0fd5368865d01537d11e53e1587",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fd5368865d01537d10e9bf12e6",
        "ProductRatePlan.Name": "2 Year Act! Premium - Annual Subscription (2 Year Contract)",
        "Product.productType__c": "PartnerActPremium",
        "Product.Name": "Act! Partner Premium",
        "Product.Id": "2c92a0fb4e86a5ff014e998957574841",
        "Product.SKU": "SKU-00000009",
        "Subscription.Id": "2c92a0fe5af9a6b9015b13985d0939e1",
        "Subscription.Name": "A-S00004451",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "7/30/18",
        "Subscription.SubscriptionStartDate": "7/30/15",
        "Subscription.TermStartDate": "7/30/16",
        "Subscription.TermEndDate": "7/30/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fb4edd70c9014ee65661bb7804",
        "Account.Name": "Windsor Corporate Suites",
        "Account.AccountNumber": "A00025449",
        "Account.resellerofRecord__c": "4000919033 AspenTech Consulting Group, Inc.",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "7/31/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "AGeaudreau@windsorcommunities.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Massachusetts",
        "BillToContact.WorkEmail": "mknapp@aspen-tech.com",
        "RatePlan.Id": "2c92a0fe5af9a6b9015b13985d9e3a03",
        "RatePlan.Name": "2 Year Act! Premium - Annual Subscription (2 Year Contract Loyalty Pricing)",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b9015b13985d9e3a04",
        "RatePlanCharge.BillingPeriod": "Two Years",
        "RatePlanCharge.Description": "2 Year Act! Premium - Annual Subscription (2 Year Contract Loyalty Pricing)",
        "RatePlanCharge.Quantity": "6",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "7/30/18",
        "ProductRatePlanCharge.Id": "2c92a0fd5368865d01537d1325db1a5f",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Loyalty",
        "ProductRatePlan.Id": "2c92a0fd5368865d01537d12736c1878",
        "ProductRatePlan.Name": "2 Year Act! Premium - Annual Subscription (2 Year Contract Loyalty Pricing)",
        "Product.productType__c": "PartnerActPremium",
        "Product.Name": "Act! Partner Premium",
        "Product.Id": "2c92a0fb4e86a5ff014e998957574841",
        "Product.SKU": "SKU-00000009",
        "Subscription.Id": "2c92a0fe5af9a6b9015b13985d0939e1",
        "Subscription.Name": "A-S00004451",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "7/30/18",
        "Subscription.SubscriptionStartDate": "7/30/15",
        "Subscription.TermStartDate": "7/30/16",
        "Subscription.TermEndDate": "7/30/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd53f4c86c01540c58c18d21ed",
        "Account.Name": "Jenco Technologies, Inc",
        "Account.AccountNumber": "A00094335",
        "Account.resellerofRecord__c": "4000919441 Effective Business Solutions",
        "Account.CreatedDate": "4/12/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "dan@jencotech.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Utah",
        "BillToContact.WorkEmail": "kvb@actcolorado.com",
        "RatePlan.Id": "2c92a0ff5af9b657015b13985e6e334d",
        "RatePlan.Name": "Silver Discount Annual",
        "RatePlanCharge.Id": "2c92a0ff5af9b657015b13985e6e334e",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "This discount is available to Act! Consultants at the Silver tier. Applying this discount will lower the subscription price but will prevent them from receiving commission. ",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "4/11/18",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014cccff9ff635c4",
        "ProductRatePlan.planType__c": "MarginDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94cc721de014cccff9d997246",
        "ProductRatePlan.Name": "Silver Discount Annual",
        "Product.productType__c": "ActMarginDiscounts",
        "Product.Name": "Act! Margin",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c823cdc",
        "Product.SKU": "SKU-00000006",
        "Subscription.Id": "2c92a0ff5af9b657015b13985e573347",
        "Subscription.Name": "A-S00032181",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "4/11/18",
        "Subscription.SubscriptionStartDate": "4/11/16",
        "Subscription.TermStartDate": "4/11/16",
        "Subscription.TermEndDate": "4/11/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd53d0349a0153d301e34b3cb8",
        "Account.Name": "Texas Catering",
        "Account.AccountNumber": "A00089357",
        "Account.commisionedSalesRep__c": "Chester.Miles@swiftpage.com",
        "Account.CreatedDate": "4/1/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "jenni@texascatering.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Texas",
        "BillToContact.WorkEmail": "jenni@texascatering.com",
        "RatePlan.Id": "2c92a0fe5af9a6b8015b1398616b29a1",
        "RatePlan.Name": "2 Year Act! Premium Cloud - Annual Subscription (2 Year Contract)",
        "RatePlanCharge.Id": "2c92a0fe5af9a6b8015b1398616b29a2",
        "RatePlanCharge.BillingPeriod": "Two Years",
        "RatePlanCharge.Description": "2 Year Act! Premium Cloud - Annual Subscription (2 Year Contract)\\n",
        "RatePlanCharge.Quantity": "7",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "4/18/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "11",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2021",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************3921",
        "ProductRatePlanCharge.Id": "2c92a0fd5368866001537d1d8fde1b6a",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fd5368866001537d1cbc65147d",
        "ProductRatePlan.Name": "2 Year Act! Premium Cloud - Annual Subscription (2 Year Contract)",
        "Product.productType__c": "ActPremiumCloud",
        "Product.Name": "Act! Premium Cloud",
        "Product.Id": "2c92a0fb4a38064e014a55db0e66551c",
        "Product.SKU": "SKU-00000002",
        "Subscription.Id": "2c92a0fe5af9a6b8015b1398614e299d",
        "Subscription.Name": "A-S00033068",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "4/18/18",
        "Subscription.SubscriptionStartDate": "4/18/16",
        "Subscription.TermStartDate": "4/18/16",
        "Subscription.TermEndDate": "4/18/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0f94d664807014d7c69fda41c61",
        "Account.Name": "CLEARONE",
        "Account.AccountNumber": "A00007417",
        "Account.renewalRep__c": "chris.clary@swiftpage.com",
        "Account.commisionedSalesRep__c": "chris.clary@swiftpage.com",
        "Account.CreatedDate": "5/22/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "diana.schroeder@clearone.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Utah",
        "BillToContact.WorkEmail": "diana.schroeder@clearone.com",
        "RatePlan.Id": "2c92a0fd5aff18a5015b139866087f7e",
        "RatePlan.Name": "2 Year Act! Premier Support (2 Year Contract)",
        "RatePlanCharge.Id": "2c92a0fd5aff18a5015b139866087f80",
        "RatePlanCharge.BillingPeriod": "Two Years",
        "RatePlanCharge.Description": "2 Year Act! Premier Support (2 year contract)",
        "RatePlanCharge.Quantity": "35",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "4/14/18",
        "ProductRatePlanCharge.Id": "2c92a0fd536886bd01537d09f5cf7856",
        "ProductRatePlan.planType__c": "Support",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fd5368866001537d08d9ad1bc3",
        "ProductRatePlan.Name": "2 Year Act! Premier Support (2 Year Contract)",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0fd5aff18a5015b139865ed7f73",
        "Subscription.Name": "A-S00032606",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "4/14/18",
        "Subscription.SubscriptionStartDate": "4/14/16",
        "Subscription.TermStartDate": "4/14/16",
        "Subscription.TermEndDate": "4/14/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff542863fd0154303532581e89",
        "Account.Name": "Westmont Hospitality Group/InnVest Hotels",
        "Account.AccountNumber": "A00097034",
        "Account.resellerofRecord__c": "4001372618 Keystroke Quality Computing, Inc.",
        "Account.renewalRep__c": "laczy.frazee@swiftpage.com",
        "Account.commisionedSalesRep__c": "laczy.frazee@swiftpage.com",
        "Account.CreatedDate": "4/19/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "patti.weir@whg.com",
        "SoldToContact.Country": "Canada",
        "SoldToContact.State": "Ontario",
        "BillToContact.WorkEmail": "kenquigley@kqc.ca",
        "RatePlan.Id": "2c92a0fd5aff18a5015b13988d670022",
        "RatePlan.Name": "2 Year Act! Premier Support - Annual (2 Year Contract)",
        "RatePlanCharge.Id": "2c92a0fd5aff18a5015b13988d670023",
        "RatePlanCharge.BillingPeriod": "Two Years",
        "RatePlanCharge.Description": "2 Year Act! Premier Support - Annual (2 Year Contract)",
        "RatePlanCharge.Quantity": "42",
        "RatePlanCharge.Version": "5",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "5/3/16",
        "ProductRatePlanCharge.Id": "2c92a0fd536886bd01537d19af074918",
        "ProductRatePlan.planType__c": "Support",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0ff535fffd301537d182bde73ca",
        "ProductRatePlan.Name": "2 Year Act! Premier Support - Annual (2 Year Contract)",
        "Product.productType__c": "PartnerActPremium",
        "Product.Name": "Act! Partner Premium",
        "Product.Id": "2c92a0fb4e86a5ff014e998957574841",
        "Product.SKU": "SKU-00000009",
        "Subscription.Id": "2c92a0fd5aff18a5015b13988d220015",
        "Subscription.Name": "A-S00033079",
        "Subscription.Status": "Active",
        "Subscription.Reseller__c": "4001372618 Keystroke Quality Computing",
        "Subscription.SubscriptionEndDate": "4/18/18",
        "Subscription.SubscriptionStartDate": "4/18/16",
        "Subscription.TermStartDate": "4/18/16",
        "Subscription.TermEndDate": "4/18/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fb4d665b80014d7c6c96fb4626",
        "Account.Name": "Bass Tool & Supply",
        "Account.AccountNumber": "A00011361",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "5/22/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "rlewis@basstool.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Texas",
        "BillToContact.WorkEmail": "rlewis@basstool.com",
        "RatePlan.Id": "2c92a0ff5af9b659015b1398a0d96cf6",
        "RatePlan.Name": "2 Year Act! Premium - Annual Subscription (2 Year Contract Business Care Bronze Migration Users)",
        "RatePlanCharge.Id": "2c92a0ff5af9b659015b1398a0d96cf7",
        "RatePlanCharge.BillingPeriod": "Two Years",
        "RatePlanCharge.Description": "2 Year Act! Premium - Annual Subscription (2 Year Contract Business Care Bronze Migration User)",
        "RatePlanCharge.Quantity": "15",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "5/26/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "10",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************0219",
        "ProductRatePlanCharge.Id": "2c92a0fc5360a65101537d06cfcb3019",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "NoSupportMigration",
        "ProductRatePlan.Id": "2c92a0ff535fffd301537d05e79d0c29",
        "ProductRatePlan.Name": "2 Year Act! Premium - Annual Subscription (2 Year Contract Business Care Bronze Migration Users)",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0ff5af9b659015b1398a0c26ceb",
        "Subscription.Name": "A-S00038620",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/26/18",
        "Subscription.SubscriptionStartDate": "5/26/16",
        "Subscription.TermStartDate": "5/26/16",
        "Subscription.TermEndDate": "5/26/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0f94d664802014d7c6ace3f74e5",
        "Account.Name": "North Bay Permit Services",
        "Account.AccountNumber": "A00007967",
        "Account.resellerofRecord__c": "4000919200 Active Business Systems",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "5/22/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "marjorie@activebizsystems.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "marjorie@activebizsystems.com",
        "RatePlan.Id": "2c92a0ff5af9b657015b1398b2c67263",
        "RatePlan.Name": "2 Year Act! Premium - Annual Subscription (2 Year Contract Business Care Bronze Additional User)",
        "RatePlanCharge.Id": "2c92a0ff5af9b657015b1398b2c67264",
        "RatePlanCharge.BillingPeriod": "Two Years",
        "RatePlanCharge.Description": "2 Year Act! Premium - Annual Subscription (2 Year Contract Business Care Bronze Additional User)",
        "RatePlanCharge.Quantity": "2",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "6/13/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "3",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************6644",
        "ProductRatePlanCharge.Id": "2c92a0fd5368866001537d045f527f34",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "NoSupport",
        "ProductRatePlan.Id": "2c92a0ff535fffd501537d037a5c5a48",
        "ProductRatePlan.Name": "2 Year Act! Premium - Annual Subscription (2 Year Contract Business Care Bronze Additional User)",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0ff5af9b657015b1398b2827255",
        "Subscription.Name": "A-S00040799",
        "Subscription.Status": "Active",
        "Subscription.Reseller__c": "4000919200 Active Business Systems ",
        "Subscription.SubscriptionEndDate": "6/13/18",
        "Subscription.SubscriptionStartDate": "6/13/16",
        "Subscription.TermStartDate": "6/13/16",
        "Subscription.TermEndDate": "6/13/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff561458f501562dcccdb35934",
        "Account.Name": "JEM",
        "Account.AccountNumber": "A00133734",
        "Account.commisionedSalesRep__c": "todd.farnham@swiftpage.com",
        "Account.CreatedDate": "7/27/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "mbriggs@jemengineering.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Maryland",
        "BillToContact.WorkEmail": "mbriggs@jemengineering.com",
        "RatePlan.Id": "2c92a0ff5af9b659015b1398e0bc7280",
        "RatePlan.Name": "2 Year Act! Cloud Premier Support - Annual (2 Year Contract)",
        "RatePlanCharge.Id": "2c92a0ff5af9b659015b1398e0bc7281",
        "RatePlanCharge.BillingPeriod": "Two Years",
        "RatePlanCharge.Description": "2 Year Act! Cloud Premier Support - Annual (2 Year Contract)",
        "RatePlanCharge.Quantity": "3",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "3/27/17",
        "RatePlanCharge.EffectiveEndDate": "7/26/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "8",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************0998",
        "ProductRatePlanCharge.Id": "2c92a0fd536886bd01537d1c4d1e6380",
        "ProductRatePlan.Id": "2c92a0fd536886bd01537d1af44b52e1",
        "ProductRatePlan.Name": "2 Year Act! Cloud Premier Support - Annual (2 Year Contract)",
        "Product.productType__c": "ActPremiumCloud",
        "Product.Name": "Act! Premium Cloud",
        "Product.Id": "2c92a0fb4a38064e014a55db0e66551c",
        "Product.SKU": "SKU-00000002",
        "Subscription.Id": "2c92a0ff5af9b659015b1398e0a0726e",
        "Subscription.Name": "A-S00046626",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "7/26/18",
        "Subscription.SubscriptionStartDate": "7/26/16",
        "Subscription.TermStartDate": "7/26/16",
        "Subscription.TermEndDate": "7/26/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd5b14d5c0015b170b6ac70d04",
        "Account.Name": "Law Offices of Vince Ward",
        "Account.AccountNumber": "A00191861",
        "Account.resellerofRecord__c": "4000919137 TMW Consulting",
        "Account.CreatedDate": "3/28/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "chris@vinceward.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "tmwilli@tmwconsulting.net",
        "RatePlan.Id": "2c92a0ff5b14d5cf015b170cced0020c",
        "RatePlan.Name": "Act! Pro Volume Discounts (10-29)",
        "RatePlanCharge.Id": "2c92a0ff5b14d5cf015b170ccede020d",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Act! Pro Volume Discounts (10-29)",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "3/28/17",
        "RatePlanCharge.EffectiveEndDate": "3/27/18",
        "ProductRatePlanCharge.Id": "2c92a0fd5316e85301531bb8de594048",
        "ProductRatePlan.planType__c": "VolumeDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fd5316e85301531baf96c02bbd",
        "ProductRatePlan.Name": "Act! Pro Volume Discounts (20-49)",
        "Product.productType__c": "ActProDiscounts",
        "Product.Name": "Act! Pro Volume Discounts",
        "Product.Id": "2c92a0fd5316e85401531bab07f12625",
        "Product.SKU": "SKU-00000017",
        "Subscription.Id": "2c92a0ff5b14d5cf015b170cce5e01f8",
        "Subscription.Name": "A-S00083311",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/27/18",
        "Subscription.SubscriptionStartDate": "3/27/17",
        "Subscription.TermStartDate": "3/27/17",
        "Subscription.TermEndDate": "3/27/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe5b1ae79e015b1f9526ed63d9",
        "Account.Name": "Seven Point Financial",
        "Account.AccountNumber": "A00192262",
        "Account.commisionedSalesRep__c": "natalie.sova@swiftpage.com",
        "Account.CreatedDate": "3/30/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "bob7pt@lpl.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Massachusetts",
        "BillToContact.WorkEmail": "bob7pt@lpl.com",
        "RatePlan.Id": "2c92a0fe5b1ae79b015b1f95fd8a25eb",
        "RatePlan.Name": "Bronze Discount Annual",
        "RatePlanCharge.Id": "2c92a0fe5b1ae79b015b1f95fd9f25ec",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "This discount is available to Act! Consultants at the Bronze tier. Applying this discount will lower the subscription price but will prevent them from receiving commission. ",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "3/30/17",
        "RatePlanCharge.EffectiveEndDate": "3/29/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "2",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2018",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************5224",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014cccffa15e35f4",
        "ProductRatePlan.planType__c": "MarginDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94cc721de014cccff9de07256",
        "ProductRatePlan.Name": "Bronze Discount Annual",
        "Product.productType__c": "ActMarginDiscounts",
        "Product.Name": "Act! Margin",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c823cdc",
        "Product.SKU": "SKU-00000006",
        "Subscription.Id": "2c92a0fe5b1ae79b015b1f95fd4325dc",
        "Subscription.Name": "A-S00083623",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/29/18",
        "Subscription.SubscriptionStartDate": "3/29/17",
        "Subscription.TermStartDate": "3/29/17",
        "Subscription.TermEndDate": "3/29/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd53c639d90153ce6498f4227e",
        "Account.Name": "Air Energy LLC",
        "Account.AccountNumber": "A00088775",
        "Account.resellerofRecord__c": "4001315924 Actability",
        "Account.renewalRep__c": "jordan.colson@swiftpage.com",
        "Account.commisionedSalesRep__c": "jordan.colson@swiftpage.com",
        "Account.CreatedDate": "3/31/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "frank@compqc.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Massachusetts",
        "BillToContact.WorkEmail": "info@actability.com",
        "RatePlan.Id": "2c92a0a75b2baa22015b31cebfab40f9",
        "RatePlan.Name": "Gold Discount Annual",
        "RatePlanCharge.Id": "2c92a0a75b2baa22015b31cebfab40fa",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "This discount is available to Act! Consultants at the Gold tier. Applying this discount will lower the subscription price but will prevent them from receiving commission. ",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "4/2/17",
        "RatePlanCharge.EffectiveEndDate": "3/30/17",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014cccff9fae35b5",
        "ProductRatePlan.planType__c": "MarginDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94cc721de014cccff9d807244",
        "ProductRatePlan.Name": "Gold Discount Annual",
        "Product.productType__c": "ActMarginDiscounts",
        "Product.Name": "Act! Margin",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c823cdc",
        "Product.SKU": "SKU-00000006",
        "Subscription.Id": "2c92a0a75b2baa22015b31cebf4340eb",
        "Subscription.Name": "A-S00030818",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/30/18",
        "Subscription.SubscriptionStartDate": "3/30/16",
        "Subscription.TermStartDate": "3/30/17",
        "Subscription.TermEndDate": "3/30/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0f94d664802014d7c69d355687e",
        "Account.Name": "SAIL VACATIONS LTD",
        "Account.AccountNumber": "A00007110",
        "Account.renewalRep__c": "Jacob.garcia@swiftpage.com",
        "Account.commisionedSalesRep__c": "Jacob.garcia@swiftpage.com",
        "Account.CreatedDate": "5/22/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "barryesii@gmail.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Florida",
        "BillToContact.WorkEmail": "barryesii@gmail.com",
        "RatePlan.Id": "2c92a0a75b2baa22015b31cee82e440a",
        "RatePlan.Name": "AEM Call List",
        "RatePlanCharge.Id": "2c92a0a75b2baa22015b31cee82e440b",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "AEM Call List",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "5",
        "RatePlanCharge.CreatedDate": "4/2/17",
        "RatePlanCharge.EffectiveEndDate": "3/28/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "5",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2018",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************5114",
        "ProductRatePlanCharge.Id": "2c92a0fe56145911015629c82cfa50ec",
        "ProductRatePlan.planType__c": "CallList",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fd5614305c015629c6fc110822",
        "ProductRatePlan.Name": "AEM Call List",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0a75b2baa22015b31cee80b4406",
        "Subscription.Name": "A-S00030450",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/28/18",
        "Subscription.SubscriptionStartDate": "3/28/16",
        "Subscription.TermStartDate": "3/28/17",
        "Subscription.TermEndDate": "3/28/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd53b1b34c0153c0194d5842fb",
        "Account.Name": "MVP Builders",
        "Account.AccountNumber": "A00086307",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "3/28/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "info@mvp-builders.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Colorado",
        "BillToContact.WorkEmail": "info@mvp-builders.com",
        "RatePlan.Id": "2c92a0a75b2baa22015b31cf2d06495b",
        "RatePlan.Name": "Act! Premium - Annual Subscription (Volume)",
        "RatePlanCharge.Id": "2c92a0a75b2baa22015b31cf2d06495c",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Entitles customer to an annual subscription of Act! Premium.",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "5",
        "RatePlanCharge.CreatedDate": "4/2/17",
        "RatePlanCharge.EffectiveEndDate": "3/29/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "4",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************2740",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014ccd31067133a3",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fb4cc7343c014ccd3104350c22",
        "ProductRatePlan.Name": "Act! Premium - Annual Subscription (Volume)",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0a75b2baa22015b31cf2ce14953",
        "Subscription.Name": "A-S00030657",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/29/18",
        "Subscription.SubscriptionStartDate": "3/29/16",
        "Subscription.TermStartDate": "3/29/17",
        "Subscription.TermEndDate": "3/29/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff53d03f6d0153d1ff2fa63acd",
        "Account.Name": "EMAX Laboratories Inc.",
        "Account.AccountNumber": "A00089166",
        "Account.commisionedSalesRep__c": "gregory.davis@swiftpage.com",
        "Account.CreatedDate": "4/1/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "CPANG@EMAXLABS.COM",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "CPANG@EMAXLABS.COM",
        "RatePlan.Id": "2c92a0a65b1b7a3e015b32f9c3c337bc",
        "RatePlan.Name": "Partner Distribution Margin Discount",
        "RatePlanCharge.Id": "2c92a0a65b1b7a3e015b32f9c3c337bd",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "This discount is available to Distributors and Resellers tier. Applying this discount will lower the subscription price but will prevent them from also receiving commission. \\n",
        "RatePlanCharge.Version": "5",
        "RatePlanCharge.CreatedDate": "4/3/17",
        "RatePlanCharge.EffectiveEndDate": "3/31/18",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c83014e234f42c0677f",
        "ProductRatePlan.planType__c": "MarginDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e234f413715b4",
        "ProductRatePlan.Name": "Partner Distribution Margin Discount",
        "Product.productType__c": "ActMarginDiscounts",
        "Product.Name": "Act! Margin",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c823cdc",
        "Product.SKU": "SKU-00000006",
        "Subscription.Id": "2c92a0a65b1b7a3e015b32f9c38037b3",
        "Subscription.Name": "A-S00030908",
        "Subscription.Status": "Active",
        "Subscription.Reseller__c": "4000681998 Insight Direct Inc.",
        "Subscription.SubscriptionEndDate": "3/31/18",
        "Subscription.SubscriptionStartDate": "3/31/16",
        "Subscription.TermStartDate": "3/31/17",
        "Subscription.TermEndDate": "3/31/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff5aad0caa015ad2d1183159c1",
        "Account.Name": "Lone Star Hazmat Response",
        "Account.AccountNumber": "A00189207",
        "Account.commisionedSalesRep__c": "danielle.depasquale@swiftpage.com",
        "Account.CreatedDate": "3/15/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "randy@lonestarhazmat.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Texas",
        "BillToContact.WorkEmail": "randy@lonestarhazmat.com",
        "RatePlan.Id": "2c92a0ff5b1bc79e015b34483dda06d2",
        "RatePlan.Name": "Act! Premium - Monthly Subscription (Volume)",
        "RatePlanCharge.Id": "2c92a0ff5b1bc79e015b34483deb06d3",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Entitles customer to a monthly subscription of Act! Premium.",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "4/3/17",
        "RatePlanCharge.EffectiveEndDate": "4/2/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "8",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************2105",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014ccd31058a336e",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fb4cc7343c014ccd3104230c20",
        "ProductRatePlan.Name": "Act! Premium - Monthly Subscription (Volume)",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0ff5b1bc79e015b34483dbf06c9",
        "Subscription.Name": "A-S00084245",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "4/2/18",
        "Subscription.SubscriptionStartDate": "4/2/17",
        "Subscription.TermStartDate": "4/2/17",
        "Subscription.TermEndDate": "4/2/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff53d03f6d0153d2d2fb3d3166",
        "Account.Name": "Nelson Design Group",
        "Account.AccountNumber": "A00089303",
        "Account.resellerofRecord__c": "4000918974 Training Solutions, Inc",
        "Account.renewalRep__c": "chris.clary@swiftpage.com",
        "Account.commisionedSalesRep__c": "chris.clary@swiftpage.com",
        "Account.CreatedDate": "4/1/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "heather.wills@nelsondesigngroup.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Arkansas",
        "BillToContact.WorkEmail": "cindy.kool@trainingsolutionsinc.com",
        "RatePlan.Id": "2c92a0fc5b1ae78b015b395cd23c1b4c",
        "RatePlan.Name": "AEM Basic - Annual",
        "RatePlanCharge.Id": "2c92a0fc5b1ae78b015b395cd23c1b4d",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "AEM User Charges",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "4/4/17",
        "RatePlanCharge.EffectiveEndDate": "3/31/18",
        "ProductRatePlanCharge.Id": "2c92a0fe5b1ae79e015b2533956b37ef",
        "ProductRatePlan.planType__c": "AemTier",
        "ProductRatePlan.planSubType__c": "BasicAnnual",
        "ProductRatePlan.Id": "2c92a0fd5b1bc784015b2531e5110806",
        "ProductRatePlan.Name": "AEM Basic - Annual",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0ff5b1bc79e015b395d1cb358d6",
        "Subscription.Name": "A-S00030949",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/31/18",
        "Subscription.SubscriptionStartDate": "3/31/16",
        "Subscription.TermStartDate": "3/31/17",
        "Subscription.TermEndDate": "3/31/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff5b1bc79e015b402e3da7142a",
        "Account.Name": "EverMed DPC",
        "Account.AccountNumber": "A00193468",
        "Account.resellerofRecord__c": "4001586315 Hayes Consulting LLC",
        "Account.CreatedDate": "4/5/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "lanz@corporategrowthconsulting.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Washington",
        "BillToContact.WorkEmail": "lanz@corporategrowthconsulting.com",
        "RatePlan.Id": "2c92a0fe5b1ae79e015b403185d314ab",
        "RatePlan.Name": "Act! Premium Cloud Plus - Annual Subscription",
        "RatePlanCharge.Id": "2c92a0fe5b1ae79e015b403185e214ad",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Act! Premium Cloud Plus Per User Charge",
        "RatePlanCharge.Quantity": "8",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "4/5/17",
        "RatePlanCharge.EffectiveEndDate": "4/4/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "1",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2021",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************0250",
        "ProductRatePlanCharge.Id": "2c92a0ff5740731d015749dc653c0ff2",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fd57407d02015749db1a687b41",
        "ProductRatePlan.Name": "Act! Premium Cloud Plus - Annual Subscription",
        "Product.productType__c": "ActPremiumCloudPlus",
        "Product.Name": "Act! Premium Cloud Plus",
        "Product.Id": "2c92a0ff5740731d015749d9dbf57d3e",
        "Product.SKU": "SKU-00000021",
        "Subscription.Id": "2c92a0fe5b1ae79e015b40318593149a",
        "Subscription.Name": "A-S00084635",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "4/4/18",
        "Subscription.SubscriptionStartDate": "4/4/17",
        "Subscription.TermStartDate": "4/4/17",
        "Subscription.TermEndDate": "4/4/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff5ba41fdf015babdac1be1515",
        "Account.Name": "DAVID SALAZAR",
        "Account.AccountNumber": "A00197009",
        "Account.commisionedSalesRep__c": "Chester.Miles@swiftpage.com",
        "Account.CreatedDate": "4/26/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "dsalazar@salazargroupinc.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "dsalazar@salazargroupinc.com",
        "RatePlan.Id": "2c92a0ff5ba41fe8015babddf5a71e97",
        "RatePlan.Name": "Handheld Contact â Annual Subscription",
        "RatePlanCharge.Id": "2c92a0ff5ba41fe8015babddf5b71e98",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Handheld Contact - Annual subscription charges",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "4/26/17",
        "RatePlanCharge.EffectiveEndDate": "4/25/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "1",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2022",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********5002",
        "ProductRatePlanCharge.Id": "2c92a0fc5b89d61a015b9e0306b206a9",
        "ProductRatePlan.planType__c": "HHCAddOn",
        "ProductRatePlan.planSubType__c": "HHCStandard",
        "ProductRatePlan.Id": "2c92a0fc5b89d61a015b9e01441103cc",
        "ProductRatePlan.Name": "Handheld Contact â Annual Subscription",
        "Product.productType__c": "HandheldContact",
        "Product.Name": "Handheld Contact",
        "Product.Id": "2c92a0ff5b89ef4b015b9df309627ed3",
        "Product.SKU": "SKU-00000029",
        "Subscription.Id": "2c92a0ff5ba41fe8015babddf52b1e82",
        "Subscription.Name": "A-S00087502",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "4/25/18",
        "Subscription.SubscriptionStartDate": "4/25/17",
        "Subscription.TermStartDate": "4/25/17",
        "Subscription.TermEndDate": "4/25/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe5b810758015b85df4065088b",
        "Account.Name": "Vida Ocupacional",
        "Account.AccountNumber": "A00195599",
        "Account.resellerofRecord__c": "4000919321 Praxi Solutions Inc.",
        "Account.commisionedSalesRep__c": "Danielle.DePasquale@swiftpage.com",
        "Account.CreatedDate": "4/19/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "vidaocupacionalsiteoficial@gmail.com",
        "SoldToContact.Country": "Brazil",
        "SoldToContact.State": "SP",
        "BillToContact.WorkEmail": "vidaocupacionalsiteoficial@gmail.com",
        "RatePlan.Id": "2c92a0fd5baeb770015bb081ec543d8c",
        "RatePlan.Name": "AEM Contact Block",
        "RatePlanCharge.Id": "2c92a0fd5baeb770015bb081ec543d8d",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "AEM Contact Block",
        "RatePlanCharge.Quantity": "2",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "4/27/17",
        "RatePlanCharge.EffectiveEndDate": "4/18/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "5",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2023",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************2918",
        "ProductRatePlanCharge.Id": "2c92a0fe56145911015629c67c544a32",
        "ProductRatePlan.planType__c": "ContactBlocks",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0ff561458f5015629c5bd3a5ddd",
        "ProductRatePlan.Name": "AEM Contact Block",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0fd5baeb770015bb081ec483d88",
        "Subscription.Name": "A-S00086324",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "4/18/18",
        "Subscription.SubscriptionStartDate": "4/18/17",
        "Subscription.TermStartDate": "4/18/17",
        "Subscription.TermEndDate": "4/18/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff5ba41fdf015ba6243b862680",
        "Account.Name": "CHARAN ASSOCIATES INC.",
        "Account.AccountNumber": "A00196606",
        "Account.commisionedSalesRep__c": "greg.stellmacher@swiftpage.com",
        "Account.CreatedDate": "4/25/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "rsmithhart@netteamtech.net",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Texas",
        "BillToContact.WorkEmail": "rsmithhart@netteamtech.net",
        "RatePlan.Id": "2c92a0ff5bb41c4d015bc4d698cc449d",
        "RatePlan.Name": "Act! Premium 2017 Update - Perpetual License",
        "RatePlanCharge.Id": "2c92a0ff5bb41c4d015bc4d698dd449e",
        "RatePlanCharge.Description": "Act! Premium 2017 Update Perpetual License",
        "RatePlanCharge.Quantity": "3",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "5/1/17",
        "RatePlanCharge.EffectiveEndDate": "5/1/17",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "1",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2022",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********2007",
        "ProductRatePlanCharge.Id": "2c92a0ff5bb41c4e015bc0eb5682293f",
        "ProductRatePlan.planType__c": "PerpetualLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fe5bb40fd7015bc0ea7ebf1cf3",
        "ProductRatePlan.Name": "Act! Premium 2017 Update - Perpetual License",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0ff5bb41c4d015bc4d698654483",
        "Subscription.Name": "A-S00088598",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "4/30/18",
        "Subscription.SubscriptionStartDate": "4/30/17",
        "Subscription.TermStartDate": "4/30/17",
        "Subscription.TermEndDate": "4/30/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe5bb40fd5015bc5898512586a",
        "Account.Name": "SwiftpageTest Erika",
        "Account.AccountNumber": "A00198360",
        "Account.CreatedDate": "5/1/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "erika1test+050117p@gmail.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Arizona",
        "BillToContact.WorkEmail": "erika1test+050117p@gmail.com",
        "RatePlan.Id": "2c92a0fe5bb40fd5015bc58985615877",
        "RatePlan.Name": "Act! Premium 2017 Update - Annual Subscription",
        "RatePlanCharge.Id": "2c92a0fe5bb40fd5015bc589856b5879",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Act! Premium - 2017 Annual Subscription",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "5/1/17",
        "RatePlanCharge.EffectiveEndDate": "4/30/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "3",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2021",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********1017",
        "ProductRatePlanCharge.Id": "2c92a0fe5bb40fd5015bc0e4627961a6",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fc5bb40fd4015bc0e2ee313e8b",
        "ProductRatePlan.Name": "Act! Premium 2017 Update - Annual Subscription",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a0fe5bb40fd5015bc58985475872",
        "Subscription.Name": "A-S00088641",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "4/30/18",
        "Subscription.SubscriptionStartDate": "4/30/17",
        "Subscription.TermStartDate": "4/30/17",
        "Subscription.TermEndDate": "4/30/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fb4d039f96014d256dd4ec032c",
        "Account.Name": "Right Stuff Equipment",
        "Account.AccountNumber": "A00002667",
        "Account.resellerofRecord__c": "4000919113 ASDS Computer Company",
        "Account.renewalRep__c": "chris.clary@swiftpage.com",
        "Account.commisionedSalesRep__c": "chris.clary@swiftpage.com",
        "Account.CreatedDate": "5/5/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "mtorres@rightstuffequipment.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Colorado",
        "BillToContact.WorkEmail": "jean@asdscomputer.com",
        "RatePlan.Id": "2c92a09a5bae9da0015bd7c774401035",
        "RatePlan.Name": "Act! Premium Loyalty Credit - Annual Subscription",
        "RatePlanCharge.Id": "2c92a09a5bae9da0015bd7c774401036",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Customers on v15-17 migrating to an annual subscription plan are entitled to this credit offset.\\n\\nAlso applies to active Business Care Silver and Gold customers. ",
        "RatePlanCharge.Quantity": "6",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "5/5/17",
        "RatePlanCharge.EffectiveEndDate": "5/4/18",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014cccffa2673616",
        "ProductRatePlan.planType__c": "LoyaltyCredit",
        "ProductRatePlan.Id": "2c92a0f94cc721de014cccff9e08725a",
        "ProductRatePlan.Name": "Act! Premium Loyalty Credit - Annual Subscription",
        "Product.Name": "Act! Premium Credits",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c903ce1",
        "Product.SKU": "SKU-00000007",
        "Subscription.Id": "2c92a09a5bae9da0015bd7c7740b101f",
        "Subscription.Name": "A-S00001636",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/4/18",
        "Subscription.SubscriptionStartDate": "5/4/15",
        "Subscription.TermStartDate": "5/4/17",
        "Subscription.TermEndDate": "5/4/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fb4d039f96014d256dd4ec032c",
        "Account.Name": "Right Stuff Equipment",
        "Account.AccountNumber": "A00002667",
        "Account.resellerofRecord__c": "4000919113 ASDS Computer Company",
        "Account.renewalRep__c": "chris.clary@swiftpage.com",
        "Account.commisionedSalesRep__c": "chris.clary@swiftpage.com",
        "Account.CreatedDate": "5/5/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "mtorres@rightstuffequipment.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Colorado",
        "BillToContact.WorkEmail": "jean@asdscomputer.com",
        "RatePlan.Id": "2c92a09a5bae9da0015bd7c774541042",
        "RatePlan.Name": "Act! Premium Perpetual License",
        "RatePlanCharge.Id": "2c92a09a5bae9da0015bd7c774541043",
        "RatePlanCharge.Description": "One time license fee that entitles customers to the current version when they cancel. Applies to new users or customers upgrading from v15-17. For active Business Care Customers use Perpetual License - Business Care Migration. \\n\\nThis item cannot be purchased by itself.",
        "RatePlanCharge.Quantity": "6",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "5/5/17",
        "RatePlanCharge.EffectiveEndDate": "5/5/15",
        "ProductRatePlanCharge.Id": "2c92a0fb4cc7343f014cccf566fb0c5d",
        "ProductRatePlan.planType__c": "PerpetualLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fb4cc7343c014cccf561eb4e1a",
        "ProductRatePlan.Name": "Act! Premium Perpetual License (Volume)",
        "Product.productType__c": "ActPremium",
        "Product.Name": "Act! Premium",
        "Product.Id": "2c92a0fb4cc73436014cccf5603578c4",
        "Product.SKU": "SKU-00000004",
        "Subscription.Id": "2c92a09a5bae9da0015bd7c7740b101f",
        "Subscription.Name": "A-S00001636",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/4/18",
        "Subscription.SubscriptionStartDate": "5/4/15",
        "Subscription.TermStartDate": "5/4/17",
        "Subscription.TermEndDate": "5/4/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fb4d039f96014d256dd4ec032c",
        "Account.Name": "Right Stuff Equipment",
        "Account.AccountNumber": "A00002667",
        "Account.resellerofRecord__c": "4000919113 ASDS Computer Company",
        "Account.renewalRep__c": "chris.clary@swiftpage.com",
        "Account.commisionedSalesRep__c": "chris.clary@swiftpage.com",
        "Account.CreatedDate": "5/5/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "mtorres@rightstuffequipment.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Colorado",
        "BillToContact.WorkEmail": "jean@asdscomputer.com",
        "RatePlan.Id": "2c92a09a5bae9da0015bd7c7746c1047",
        "RatePlan.Name": "Act! Premium Loyalty Credit - Perpetual License",
        "RatePlanCharge.Id": "2c92a09a5bae9da0015bd7c7746c1048",
        "RatePlanCharge.Description": "Customers on v15-17 migrating to a perpetual license plan are entitled to this credit offset.\\n\\nThis does not apply to Business Care migrations.",
        "RatePlanCharge.Quantity": "6",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "5/5/17",
        "RatePlanCharge.EffectiveEndDate": "5/5/15",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014cccffa35f364d",
        "ProductRatePlan.planType__c": "LoyaltyCredit",
        "ProductRatePlan.Id": "2c92a0f94cc721de014cccff9e2e7261",
        "ProductRatePlan.Name": "Act! Premium Loyalty Credit - Perpetual License",
        "Product.Name": "Act! Premium Credits",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c903ce1",
        "Product.SKU": "SKU-00000007",
        "Subscription.Id": "2c92a09a5bae9da0015bd7c7740b101f",
        "Subscription.Name": "A-S00001636",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/4/18",
        "Subscription.SubscriptionStartDate": "5/4/15",
        "Subscription.TermStartDate": "5/4/17",
        "Subscription.TermEndDate": "5/4/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fb4d039f96014d256dd4ec032c",
        "Account.Name": "Right Stuff Equipment",
        "Account.AccountNumber": "A00002667",
        "Account.resellerofRecord__c": "4000919113 ASDS Computer Company",
        "Account.renewalRep__c": "chris.clary@swiftpage.com",
        "Account.commisionedSalesRep__c": "chris.clary@swiftpage.com",
        "Account.CreatedDate": "5/5/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "mtorres@rightstuffequipment.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Colorado",
        "BillToContact.WorkEmail": "jean@asdscomputer.com",
        "RatePlan.Id": "2c92a09a5bae9da0015bd7c77485104e",
        "RatePlan.Name": "Gold Discount Annual",
        "RatePlanCharge.Id": "2c92a09a5bae9da0015bd7c77485104f",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "This discount is available to Act! Consultants at the Gold tier. Applying this discount will lower the subscription price but will prevent them from receiving commission. ",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "5/5/17",
        "RatePlanCharge.EffectiveEndDate": "5/4/18",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014cccffa03e35d0",
        "ProductRatePlan.planType__c": "MarginDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94cc721de014cccff9dad724a",
        "ProductRatePlan.Name": "Gold Discount Annual (Legacy Tier)",
        "Product.productType__c": "ActMarginDiscounts",
        "Product.Name": "Act! Margin",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c823cdc",
        "Product.SKU": "SKU-00000006",
        "Subscription.Id": "2c92a09a5bae9da0015bd7c7740b101f",
        "Subscription.Name": "A-S00001636",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/4/18",
        "Subscription.SubscriptionStartDate": "5/4/15",
        "Subscription.TermStartDate": "5/4/17",
        "Subscription.TermEndDate": "5/4/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0f84d0393c6014d251f1d881295",
        "Account.Name": "CHERYL CORSON DESIGN",
        "Account.AccountNumber": "A00002602",
        "Account.renewalRep__c": "jacob.garcia@swiftpage.com",
        "Account.commisionedSalesRep__c": "jacob.garcia@swiftpage.com",
        "Account.CreatedDate": "5/5/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "CHERYL@CHERYLCORSON.COM",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Maryland",
        "BillToContact.WorkEmail": "CHERYL@CHERYLCORSON.COM",
        "RatePlan.Id": "2c92a0a65baeaa08015bd7c772584a1d",
        "RatePlan.Name": "Act! Premium Cloud - Annual Subscription",
        "RatePlanCharge.Id": "2c92a0a65baeaa08015bd7c772584a1e",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Entitles customer to an annual subscription of Act! Premium Cloud.",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "5/5/17",
        "RatePlanCharge.EffectiveEndDate": "5/4/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "1",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2021",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************8522",
        "ProductRatePlanCharge.Id": "2c92a0fb4cc7343c014cccf9e25b6a14",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fb4cc7343c014cccf9e0fc69ee",
        "ProductRatePlan.Name": "Act! Premium Cloud - Annual Subscription (Volume)",
        "Product.productType__c": "ActPremiumCloud",
        "Product.Name": "Act! Premium Cloud",
        "Product.Id": "2c92a0fb4a38064e014a55db0e66551c",
        "Product.SKU": "SKU-00000002",
        "Subscription.Id": "2c92a0a65baeaa08015bd7c7722e4a12",
        "Subscription.Name": "A-S00001626",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/4/18",
        "Subscription.SubscriptionStartDate": "5/4/15",
        "Subscription.TermStartDate": "5/4/17",
        "Subscription.TermEndDate": "5/4/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe5bb40fd5015be95746611340",
        "Account.Name": "CallStar.net",
        "Account.AccountNumber": "A00199719",
        "Account.commisionedSalesRep__c": "greg.stellmacher@swiftpage.com",
        "Account.CreatedDate": "5/8/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "callstar.net@gmail.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Florida",
        "BillToContact.WorkEmail": "callstar.net@gmail.com",
        "RatePlan.Id": "2c92a0ff5bb41c4e015be95cb29609ae",
        "RatePlan.Name": "AEM Team",
        "RatePlanCharge.Id": "2c92a0ff5bb41c4e015be95cb2a309af",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "AEM User Charges",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "5/8/17",
        "RatePlanCharge.EffectiveEndDate": "5/7/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "2",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************3227",
        "ProductRatePlanCharge.Id": "2c92a0ff561458f5015629c95c447c27",
        "ProductRatePlan.planType__c": "AemTier",
        "ProductRatePlan.planSubType__c": "Team",
        "ProductRatePlan.Id": "2c92a0fd5614305c015629c8a0af7f89",
        "ProductRatePlan.Name": "AEM Team",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0ff5bb41c4e015be95cb25809a4",
        "Subscription.Name": "A-S00089697",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/7/18",
        "Subscription.SubscriptionStartDate": "5/7/17",
        "Subscription.TermStartDate": "5/7/17",
        "Subscription.TermEndDate": "5/7/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fb4d039f96014d4495e3ab0107",
        "Account.Name": "United Mechanical",
        "Account.AccountNumber": "A00003732",
        "Account.renewalRep__c": "Jacob.garcia@swiftpage.com",
        "Account.commisionedSalesRep__c": "Jacob.garcia@swiftpage.com",
        "Account.CreatedDate": "5/11/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "nwalker@uaccontrols.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "North Carolina",
        "BillToContact.WorkEmail": "nwalker@uaccontrols.com",
        "RatePlan.Id": "2c92a0a65bec1ac9015bfbc9c0671922",
        "RatePlan.Name": "Act! Premium Business Care Bronze Loyalty Credit - Annual Subscription",
        "RatePlanCharge.Id": "2c92a0a65bec1ac9015bfbc9c0671923",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "This credit ONLY applies to current Business Care Bronze Users migrating to an Annual Subscription. This plan removes entitlement to standard support. ",
        "RatePlanCharge.Quantity": "8",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "5/12/17",
        "RatePlanCharge.EffectiveEndDate": "5/11/17",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "6",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2018",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************6382",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014cccffa2dd362a",
        "ProductRatePlan.planType__c": "LoyaltyCredit",
        "ProductRatePlan.Id": "2c92a0f94cc721de014cccff9e1b725e",
        "ProductRatePlan.Name": "Act! Premium Business Care Bronze Loyalty Credit - Annual Subscription",
        "Product.Name": "Act! Premium Credits",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c903ce1",
        "Product.SKU": "SKU-00000007",
        "Subscription.Id": "2c92a0a65bec1ac9015bfbc9bff318ed",
        "Subscription.Name": "A-S00001816",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/11/18",
        "Subscription.SubscriptionStartDate": "5/11/15",
        "Subscription.TermStartDate": "5/11/17",
        "Subscription.TermEndDate": "5/11/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0f94d03939e014d4e5f302a25d8",
        "Account.Name": "Vantaggio HR Ltd",
        "Account.AccountNumber": "A00004194",
        "Account.commisionedSalesRep__c": "quinn.thomson@swiftpage.com",
        "Account.CreatedDate": "5/13/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "accounting@vantaggiohr.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "accounting@vantaggiohr.com",
        "RatePlan.Id": "2c92a0a65bec1ac9015c00edcce37c0a",
        "RatePlan.Name": "Act! Premium Loyalty Credit - Monthly Subscription",
        "RatePlanCharge.Id": "2c92a0a65bec1ac9015c00edcce37c0b",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Customers on v15-17 migrating to a monthly subscription plan are entitled to this credit offset.\\n\\nAlso applies to active Business Care Silver and Gold customers. ",
        "RatePlanCharge.Quantity": "13",
        "RatePlanCharge.Version": "3",
        "RatePlanCharge.CreatedDate": "5/13/17",
        "RatePlanCharge.EffectiveEndDate": "5/12/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "7",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************4608",
        "ProductRatePlanCharge.Id": "2c92a0f94cc721de014cccffa1ef3600",
        "ProductRatePlan.planType__c": "LoyaltyCredit",
        "ProductRatePlan.Id": "2c92a0f94cc721de014cccff9df77258",
        "ProductRatePlan.Name": "Act! Premium Loyalty Credit - Monthly Subscription",
        "Product.Name": "Act! Premium Credits",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c903ce1",
        "Product.SKU": "SKU-00000007",
        "Subscription.Id": "2c92a0a65bec1ac9015c00edcc9d7bf0",
        "Subscription.Name": "A-S00001889",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/12/18",
        "Subscription.SubscriptionStartDate": "5/12/15",
        "Subscription.TermStartDate": "5/12/17",
        "Subscription.TermEndDate": "5/12/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe5bec1946015c0d04517c6da6",
        "Account.Name": "Nimlok Louisiana",
        "Account.AccountNumber": "A00200681",
        "Account.commisionedSalesRep__c": "greg.stellmacher@swiftpage.com",
        "Account.CreatedDate": "5/15/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "mitchell@nimlok-louisiana.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Louisiana",
        "BillToContact.WorkEmail": "mitchell@nimlok-louisiana.com",
        "RatePlan.Id": "2c92a0fd5c1a0f76015c1d0813585742",
        "RatePlan.Name": "AEM Call List (Team)",
        "RatePlanCharge.Id": "2c92a0fd5c1a0f76015c1d0813585743",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "AEM Call List (Team)",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "5/18/17",
        "RatePlanCharge.EffectiveEndDate": "5/14/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "5",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********1031",
        "ProductRatePlanCharge.Id": "2c92a0fd561430c3015629c52a7e2d71",
        "ProductRatePlan.planType__c": "CallList",
        "ProductRatePlan.planSubType__c": "Free",
        "ProductRatePlan.Id": "2c92a0ff561458f5015629c42d3b5412",
        "ProductRatePlan.Name": "AEM Call List (Team)",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0fd5c1a0f76015c1d0813385739",
        "Subscription.Name": "A-S00090463",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/14/18",
        "Subscription.SubscriptionStartDate": "5/14/17",
        "Subscription.TermStartDate": "5/14/17",
        "Subscription.TermEndDate": "5/14/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd54c774940154cac5c7541796",
        "Account.Name": "Amigos del Hospital de Ninos",
        "Account.AccountNumber": "A00107013",
        "Account.resellerofRecord__c": "4000919321 Praxi Solutions Inc.",
        "Account.renewalRep__c": "Laczy.Frazee@swiftpage.com",
        "Account.commisionedSalesRep__c": "Laczy.Frazee@swiftpage.com",
        "Account.CreatedDate": "5/19/16",
        "Account.Currency": "USD",
        "SoldToContact.Country": "Venezuela",
        "BillToContact.WorkEmail": "orders.latam@praxisol.com",
        "RatePlan.Id": "2c92a09a5c1d5009015c1fd66cfd70ad",
        "RatePlan.Name": "Partner Praxi Margin Discount",
        "RatePlanCharge.Id": "2c92a09a5c1d5009015c1fd66cfd70ae",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "This discount is available to Partners at the Praxi tier. Applying this discount will lower the subscription price but will prevent them from also receiving commission. ",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "5/19/17",
        "RatePlanCharge.EffectiveEndDate": "5/18/17",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c83014e234f4324678b",
        "ProductRatePlan.planType__c": "MarginDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e234f415315b6",
        "ProductRatePlan.Name": "Partner Praxi Margin Discount",
        "Product.productType__c": "ActMarginDiscounts",
        "Product.Name": "Act! Margin",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c823cdc",
        "Product.SKU": "SKU-00000006",
        "Subscription.Id": "2c92a09a5c1d5009015c1fd66cdc70a0",
        "Subscription.Name": "A-S00037462",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/18/18",
        "Subscription.SubscriptionStartDate": "5/18/16",
        "Subscription.TermStartDate": "5/18/17",
        "Subscription.TermEndDate": "5/18/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff54cd4ae40154de3ee9aa4a59",
        "Account.Name": "Australian Biosearch",
        "Account.AccountNumber": "A00107973",
        "Account.resellerofRecord__c": "411062925 ACT TODAY PTY LTD",
        "Account.renewalRep__c": "greg.weimer@swiftpage.com",
        "Account.commisionedSalesRep__c": "greg.weimer@swiftpage.com",
        "Account.CreatedDate": "5/23/16",
        "Account.Currency": "AUD",
        "SoldToContact.WorkEmail": "mark@aust-biosearch.com.au",
        "SoldToContact.Country": "Australia",
        "SoldToContact.State": "WA",
        "BillToContact.WorkEmail": "orders@acttoday.com.au",
        "RatePlan.Id": "2c92a0a65c2084eb015c346ea6aa377d",
        "RatePlan.Name": "Partner Act! Today Margin Discount",
        "RatePlanCharge.Id": "2c92a0a65c2084eb015c346ea6aa377e",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "This discount is available to Act! Consultants at the Act! Today tier. Applying this discount will lower the subscription price but will prevent them from also receiving commission. \\n",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "5/23/17",
        "RatePlanCharge.EffectiveEndDate": "1/12/17",
        "ProductRatePlanCharge.Id": "2c92a0fb4e051c83014e234f442367a3",
        "ProductRatePlan.planType__c": "MarginDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0f94e050e2e014e234f418915ba",
        "ProductRatePlan.Name": "Partner Act! Today Margin Discount",
        "Product.productType__c": "ActMarginDiscounts",
        "Product.Name": "Act! Margin",
        "Product.Id": "2c92a0fb4cc7343f014cccff9c823cdc",
        "Product.SKU": "SKU-00000006",
        "Subscription.Id": "2c92a0a65c2084eb015c346ea6893773",
        "Subscription.Name": "A-S00037834",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/22/18",
        "Subscription.SubscriptionStartDate": "5/22/16",
        "Subscription.TermStartDate": "5/22/17",
        "Subscription.TermEndDate": "5/22/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fc5399e99201539fe0a11a74db",
        "Account.Name": "Bushong Restaurant Equipment",
        "Account.AccountNumber": "A00083056",
        "Account.CreatedDate": "3/22/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "nmay@bushongequipment.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Ohio",
        "BillToContact.WorkEmail": "nmay@bushongequipment.com",
        "RatePlan.Id": "2c92a0fe5c3e2c7c015c4068190c6dc6",
        "RatePlan.Name": "Act! Password Reset",
        "RatePlanCharge.Id": "2c92a0fe5c3e2c7c015c4068190c6dc7",
        "RatePlanCharge.Description": "Act! Password Reset Charge",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "5/25/17",
        "RatePlanCharge.EffectiveEndDate": "5/25/17",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "7",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2017",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************2493",
        "ProductRatePlanCharge.Id": "2c92a0fe59b55c3d0159d255861f5bc9",
        "ProductRatePlan.Id": "2c92a0fe59b55c3c0159d254819e65fb",
        "ProductRatePlan.Name": "Act! Password Reset",
        "Product.Name": "Miscellaneous",
        "Product.Id": "2c92a0fe59b55c3d0159d253fb2b5250",
        "Product.SKU": "SKU-00000025",
        "Subscription.Id": "2c92a0fd5c3e3f21015c40682eb732d6",
        "Subscription.Name": "A-S00029575",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/21/18",
        "Subscription.SubscriptionStartDate": "3/21/16",
        "Subscription.TermStartDate": "3/21/17",
        "Subscription.TermEndDate": "3/21/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff5b1bc79e015b204735c81b35",
        "Account.Name": "California Land & Associates",
        "Account.AccountNumber": "A00192324",
        "Account.resellerofRecord__c": "4000919342 Tech Benders LLC",
        "Account.CreatedDate": "3/30/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "skiprollf@gmail.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "carol@crmTotalSolutions.com",
        "RatePlan.Id": "2c92a0fd5c633d41015c63b042fc35d3",
        "RatePlan.Name": "AEM Call List - Annual",
        "RatePlanCharge.Id": "2c92a0fd5c633d41015c63b042fc35d4",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "AEM Call List",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "6/1/17",
        "RatePlanCharge.EffectiveEndDate": "3/29/18",
        "ProductRatePlanCharge.Id": "2c92a0fe5b1ae79e015b2500b68f2d6a",
        "ProductRatePlan.planType__c": "CallList",
        "ProductRatePlan.planSubType__c": "BaseAnnual",
        "ProductRatePlan.Id": "2c92a0ff5b1bc7a2015b24fe6f347a88",
        "ProductRatePlan.Name": "AEM Call List - Annual",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0fe5c632e66015c63b065c74882",
        "Subscription.Name": "A-S00083684",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/29/18",
        "Subscription.SubscriptionStartDate": "3/29/17",
        "Subscription.TermStartDate": "3/29/17",
        "Subscription.TermEndDate": "3/29/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff5b1bc79e015b204735c81b35",
        "Account.Name": "California Land & Associates",
        "Account.AccountNumber": "A00192324",
        "Account.resellerofRecord__c": "4000919342 Tech Benders LLC",
        "Account.CreatedDate": "3/30/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "skiprollf@gmail.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "California",
        "BillToContact.WorkEmail": "carol@crmTotalSolutions.com",
        "RatePlan.Id": "2c92a0fe5c632e66015c63b066114898",
        "RatePlan.Name": "AEM Pro - Annual",
        "RatePlanCharge.Id": "2c92a0fe5c632e66015c63b066114899",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "AEM User Charges",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "6/1/17",
        "RatePlanCharge.EffectiveEndDate": "3/29/18",
        "ProductRatePlanCharge.Id": "2c92a0fe5b1ae7a9015b24a4491c463f",
        "ProductRatePlan.planType__c": "AemTier",
        "ProductRatePlan.planSubType__c": "ProAnnual",
        "ProductRatePlan.Id": "2c92a0fe5b1ae79b015b24a22b6d673d",
        "ProductRatePlan.Name": "AEM Pro - Annual",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0fe5c632e66015c63b065c74882",
        "Subscription.Name": "A-S00083684",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/29/18",
        "Subscription.SubscriptionStartDate": "3/29/17",
        "Subscription.TermStartDate": "3/29/17",
        "Subscription.TermEndDate": "3/29/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0f94d664807014d7c69a61d159a",
        "Account.Name": "The Bank of Canton",
        "Account.AccountNumber": "A00006862",
        "Account.resellerofRecord__c": "4000919055 Allied Financial Software, Inc.",
        "Account.CreatedDate": "5/22/15",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "dcheevers@thebankofcanton.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Massachusetts",
        "BillToContact.WorkEmail": "scott@software4advisors.com",
        "RatePlan.Id": "2c92a0ff5c63e3b3015c899762ba0c0e",
        "RatePlan.Name": "2 Year Act! Premium 2017 Update- Annual Subscription (2 Year Contract)",
        "RatePlanCharge.Id": "2c92a0ff5c63e3b3015c899762c70c0f",
        "RatePlanCharge.BillingPeriod": "Two Years",
        "RatePlanCharge.Description": "2 Year Act! Premium - Annual Subscription (2 Year Contract) after May 1st 2017 update",
        "RatePlanCharge.Quantity": "5",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "6/8/17",
        "RatePlanCharge.EffectiveEndDate": "6/7/18",
        "ProductRatePlanCharge.Id": "2c92a0fd5bb41c3b015bc4848fdf5bfd",
        "ProductRatePlan.planType__c": "UserLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fe5bb40fd5015bc4838e296227",
        "ProductRatePlan.Name": "2 Year Act! Premium 2017 Update- Annual Subscription (2 Year Contract)",
        "Product.productType__c": "PartnerActPremium",
        "Product.Name": "Act! Partner Premium",
        "Product.Id": "2c92a0fb4e86a5ff014e998957574841",
        "Product.SKU": "SKU-00000009",
        "Subscription.Id": "2c92a0ff5c63e3b3015c899762870c04",
        "Subscription.Name": "A-S00093165",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "6/7/18",
        "Subscription.SubscriptionStartDate": "6/7/17",
        "Subscription.TermStartDate": "6/7/17",
        "Subscription.TermEndDate": "6/7/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd5d2b95a5015d2dee8cf57a14",
        "Account.Name": "AEM Employee Account Warren",
        "Account.AccountNumber": "A00208618",
        "Account.CreatedDate": "7/10/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "Warren.Meredith@swiftpage.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Arizona",
        "BillToContact.WorkEmail": "Warren.Meredith@swiftpage.com",
        "RatePlan.Id": "2c92a0fe5d2b6948015d2e0f24b037a5",
        "RatePlan.Name": "AEM Team - Annual",
        "RatePlanCharge.Id": "2c92a0fe5d2b6948015d2e0f24b037a6",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "AEM User Charges",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "7/10/17",
        "RatePlanCharge.EffectiveEndDate": "7/9/18",
        "ProductRatePlanCharge.Id": "2c92a0fc5b1ae78b015b2502de0a2ab3",
        "ProductRatePlan.planType__c": "AemTier",
        "ProductRatePlan.planSubType__c": "TeamAnnual",
        "ProductRatePlan.Id": "2c92a0fe5b1ae79e015b250184443016",
        "ProductRatePlan.Name": "AEM Team - Annual",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0fe5d2b694b015d2e0f7a5e1e17",
        "Subscription.Name": "A-S00096562",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "7/9/18",
        "Subscription.SubscriptionStartDate": "7/9/17",
        "Subscription.TermStartDate": "7/9/17",
        "Subscription.TermEndDate": "7/9/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fd5d2b95a5015d2dee8cf57a14",
        "Account.Name": "AEM Employee Account Warren",
        "Account.AccountNumber": "A00208618",
        "Account.CreatedDate": "7/10/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "Warren.Meredith@swiftpage.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Arizona",
        "BillToContact.WorkEmail": "Warren.Meredith@swiftpage.com",
        "RatePlan.Id": "2c92a0fe5d2b694b015d2e0f7a7a1e1b",
        "RatePlan.Name": "AEM Call List (Team) - Annual",
        "RatePlanCharge.Id": "2c92a0fe5d2b694b015d2e0f7a7a1e1c",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "AEM Call List (Team)",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "7/10/17",
        "RatePlanCharge.EffectiveEndDate": "7/9/17",
        "ProductRatePlanCharge.Id": "2c92a0ff5b1bc7a2015b24f4a8cd6f9d",
        "ProductRatePlan.planType__c": "CallList",
        "ProductRatePlan.planSubType__c": "FreeAnnual",
        "ProductRatePlan.Id": "2c92a0fd5b1bc784015b24f2b86a5d72",
        "ProductRatePlan.Name": "AEM Call List (Team) - Annual",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0fe5d2b694b015d2e0f7a5e1e17",
        "Subscription.Name": "A-S00096562",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "7/9/18",
        "Subscription.SubscriptionStartDate": "7/9/17",
        "Subscription.TermStartDate": "7/9/17",
        "Subscription.TermEndDate": "7/9/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0ff5d2b8649015d3da556db201c",
        "Account.Name": "Jessica Hausknecht",
        "Account.AccountNumber": "A00209191",
        "Account.commisionedSalesRep__c": "danielle.depasquale@swiftpage.com",
        "Account.CreatedDate": "7/13/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "jhausknecht@prad.net",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Maryland",
        "BillToContact.WorkEmail": "jhausknecht@prad.net",
        "RatePlan.Id": "2c92a0fc5d2b5b45015d3dad4fc60fb1",
        "RatePlan.Name": "Handheld Contact Secure Plus - Annual Subscription",
        "RatePlanCharge.Id": "2c92a0fc5d2b5b45015d3dad4fd80fb6",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Handheld Contact Secure Plus - Annual subscription charges",
        "RatePlanCharge.Quantity": "2",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "7/13/17",
        "RatePlanCharge.EffectiveEndDate": "7/12/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "6",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2022",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********4014",
        "ProductRatePlanCharge.Id": "2c92a0ff5b89ef4b015b9e0058a312cf",
        "ProductRatePlan.planType__c": "HHCAddOn",
        "ProductRatePlan.planSubType__c": "HHCSecurePlus",
        "ProductRatePlan.Id": "2c92a0ff5b89ef49015b9df4015338b7",
        "ProductRatePlan.Name": "Handheld Contact Secure Plus - Annual Subscription",
        "Product.productType__c": "HandheldContact",
        "Product.Name": "Handheld Contact",
        "Product.Id": "2c92a0ff5b89ef4b015b9df309627ed3",
        "Product.SKU": "SKU-00000029",
        "Subscription.Id": "2c92a0fc5d2b5b45015d3dad4f540f93",
        "Subscription.Name": "A-S00096949",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "7/12/18",
        "Subscription.SubscriptionStartDate": "7/12/17",
        "Subscription.TermStartDate": "7/12/17",
        "Subscription.TermEndDate": "7/12/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe5a4b7434015a4d1e82730ef4",
        "Account.Name": "HPW Incorporated ",
        "Account.AccountNumber": "A00184512",
        "Account.commisionedSalesRep__c": "Chester.miles@swiftpage.com",
        "Account.CreatedDate": "2/17/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "johng@hpwincorporated.com",
        "SoldToContact.Country": "Canada",
        "SoldToContact.State": "Ontario",
        "BillToContact.WorkEmail": "johng@hpwincorporated.com",
        "RatePlan.Id": "2c92a0fe5d704812015d7b1a87a06644",
        "RatePlan.Name": "Act! Pro Volume Discounts (30-99)",
        "RatePlanCharge.Id": "2c92a0fe5d704812015d7b1a87b26645",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Act! Pro Volume Discounts (30-99)",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "7/25/17",
        "RatePlanCharge.EffectiveEndDate": "7/24/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "10",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************6705",
        "ProductRatePlanCharge.Id": "2c92a0ff5316fd0401531bb83cee212c",
        "ProductRatePlan.planType__c": "VolumeDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0ff5316fd0401531bb016250dbe",
        "ProductRatePlan.Name": "Act! Pro Volume Discounts (50-99)",
        "Product.productType__c": "ActProDiscounts",
        "Product.Name": "Act! Pro Volume Discounts",
        "Product.Id": "2c92a0fd5316e85401531bab07f12625",
        "Product.SKU": "SKU-00000017",
        "Subscription.Id": "2c92a0fe5d704812015d7b1a87426635",
        "Subscription.Name": "A-S00098182",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "7/24/18",
        "Subscription.SubscriptionStartDate": "7/24/17",
        "Subscription.TermStartDate": "7/24/17",
        "Subscription.TermEndDate": "7/24/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe560d310401560f4f20af0eae",
        "Account.Name": "Avant Garde Technologies, Inc.",
        "Account.AccountNumber": "A00131416",
        "Account.commisionedSalesRep__c": "Chester.Miles@Swiftpage.com",
        "Account.CreatedDate": "7/21/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "neil@agt1.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "New Jersey",
        "BillToContact.WorkEmail": "neil@agt1.com",
        "RatePlan.Id": "2c92a0fe5e1dc06a015e3fd91b1b18e5",
        "RatePlan.Name": "Act! Pro Volume Discounts (100-499)",
        "RatePlanCharge.Id": "2c92a0fe5e1dc06a015e3fd91b1b18e6",
        "RatePlanCharge.BillingPeriod": "Month",
        "RatePlanCharge.Description": "Act! Pro Volume Discounts - To get him down to loyalty pricing honoring.",
        "RatePlanCharge.Version": "4",
        "RatePlanCharge.CreatedDate": "9/1/17",
        "RatePlanCharge.EffectiveEndDate": "7/25/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "4",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2020",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********1004",
        "ProductRatePlanCharge.Id": "2c92a0fd5316e85301531bb74d7a3e09",
        "ProductRatePlan.planType__c": "VolumeDiscount",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0ff5316fcff01531bb0dd3977a8",
        "ProductRatePlan.Name": "Act! Pro Volume Discounts (100+)",
        "Product.productType__c": "ActProDiscounts",
        "Product.Name": "Act! Pro Volume Discounts",
        "Product.Id": "2c92a0fd5316e85401531bab07f12625",
        "Product.SKU": "SKU-00000017",
        "Subscription.Id": "2c92a0fe5e1dc06a015e3fd91b0218e1",
        "Subscription.Name": "A-S00045836",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "7/25/18",
        "Subscription.SubscriptionStartDate": "7/25/16",
        "Subscription.TermStartDate": "7/25/17",
        "Subscription.TermEndDate": "7/25/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe5aacfac0015acdd3648c2f91",
        "Account.Name": "Brittany roberts AEM test",
        "Account.AccountNumber": "A00188997",
        "Account.CreatedDate": "3/14/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "BrittanyPEmarketingdemo@gmail.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Arizona",
        "BillToContact.WorkEmail": "BrittanyPEmarketingdemo@gmail.com",
        "RatePlan.Id": "2c92a0fe5e1dc06a015e3fe495520733",
        "RatePlan.Name": "AEM Contact Block - Annual",
        "RatePlanCharge.Id": "2c92a0fe5e1dc06a015e3fe495520734",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "AEM Contact Block",
        "RatePlanCharge.Quantity": "43",
        "RatePlanCharge.Version": "2",
        "RatePlanCharge.CreatedDate": "9/1/17",
        "RatePlanCharge.EffectiveEndDate": "8/28/17",
        "ProductRatePlanCharge.Id": "2c92a0ff5b1bc79e015b24fa231c49f9",
        "ProductRatePlan.planType__c": "ContactBlocks",
        "ProductRatePlan.planSubType__c": "BaseAnnual",
        "ProductRatePlan.Id": "2c92a0fc5b1ae78b015b24f76ba53edc",
        "ProductRatePlan.Name": "AEM Contact Block - Annual",
        "Product.productType__c": "ActEmarketing",
        "Product.Name": "Act! eMarketing",
        "Product.Id": "2c92a0fe56145911015629c3b07c3117",
        "Product.SKU": "SKU-00000019",
        "Subscription.Id": "2c92a0fe5e1dc06a015e3fe494f50725",
        "Subscription.Name": "A-S00081136",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "3/13/18",
        "Subscription.SubscriptionStartDate": "3/13/17",
        "Subscription.TermStartDate": "3/13/17",
        "Subscription.TermEndDate": "3/13/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe5c7d81a8015c88e896340efc",
        "Account.Name": "Custom Air Products & Services Inc",
        "Account.AccountNumber": "A00204169",
        "Account.commisionedSalesRep__c": "natalie.sova@swiftpage.com",
        "Account.CreatedDate": "6/8/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "susan.archer@customairproducts.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Texas",
        "BillToContact.WorkEmail": "susan.archer@customairproducts.com",
        "RatePlan.Id": "2c92a0ff5e1dcf15015e4472fa635735",
        "RatePlan.Name": "Act! Cloud Premier Support - Annual",
        "RatePlanCharge.Id": "2c92a0ff5e1dcf15015e4472fa635736",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "Act! Cloud Premier Support - Annual",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "9/2/17",
        "RatePlanCharge.EffectiveEndDate": "6/7/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "4",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2022",
        "DefaultPaymentMethod.CreditCardMaskNumber": "***********1385",
        "ProductRatePlanCharge.Id": "2c92a0fe5d8298e1015da0b894b87e1c",
        "ProductRatePlan.planType__c": "Support",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0fe5d8298e1015da0b6d5d377cb",
        "ProductRatePlan.Name": "Act! Cloud Premier Support - Annual",
        "Product.productType__c": "ActPremiumCloud",
        "Product.Name": "Act! Premium Cloud",
        "Product.Id": "2c92a0fb4a38064e014a55db0e66551c",
        "Product.SKU": "SKU-00000002",
        "Subscription.Id": "2c92a0ff5e1dcf15015e4472fa55572d",
        "Subscription.Name": "A-S00093131",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "6/7/18",
        "Subscription.SubscriptionStartDate": "6/7/17",
        "Subscription.TermStartDate": "6/7/17",
        "Subscription.TermEndDate": "6/7/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fc54cd3f980154deb357605efd",
        "Account.Name": "Kendle Trucking Inc.",
        "Account.AccountNumber": "A00108048",
        "Account.commisionedSalesRep__c": "Chester.Miles@Swiftpage.com",
        "Account.CreatedDate": "5/23/16",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "jeff@kendletrucking.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Oregon",
        "BillToContact.WorkEmail": "jeff@kendletrucking.com",
        "RatePlan.Id": "2c92a0fd60203d27016037a1eeef61ab",
        "RatePlan.Name": "Act! Pro V20 - New License",
        "RatePlanCharge.Id": "2c92a0fd60203d27016037a1eeef61ac",
        "RatePlanCharge.Description": "Act! Pro V20 - New License",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "12/8/17",
        "RatePlanCharge.EffectiveEndDate": "12/8/17",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "3",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2019",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************4875",
        "ProductRatePlanCharge.Id": "2c92a0fe5eef37f5015f021a21845126",
        "ProductRatePlan.planType__c": "PerpetualLicense",
        "ProductRatePlan.planSubType__c": "Base",
        "ProductRatePlan.Id": "2c92a0ff5eef8272015f02190eae302d",
        "ProductRatePlan.Name": "Act! Pro V20 - New License",
        "Product.productType__c": "ActProV20",
        "Product.Name": "Act! Pro V20",
        "Product.Description": "Act! Pro V20",
        "Product.Id": "2c92a0fe5eef37f5015f021331e305d8",
        "Product.SKU": "SKU-00000035",
        "Subscription.Id": "2c92a0fe6034d486016037a22f10578f",
        "Subscription.Name": "A-S00037867",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "5/22/18",
        "Subscription.SubscriptionStartDate": "5/22/16",
        "Subscription.TermStartDate": "5/22/17",
        "Subscription.TermEndDate": "5/22/18",
        "Subscription.AutoRenew": "true"
    },
    {
        "Account.Id": "2c92a0fe5c63f1b1015c7857a0ee7b9a",
        "Account.Name": "Boston Document Systems",
        "Account.AccountNumber": "A00203493",
        "Account.CreatedDate": "6/5/17",
        "Account.Currency": "USD",
        "SoldToContact.WorkEmail": "admin@onitts.com",
        "SoldToContact.Country": "United States",
        "SoldToContact.State": "Connecticut",
        "BillToContact.WorkEmail": "admin@onitts.com",
        "RatePlan.Id": "2c92a0fc610e738701610ef41f0d2171",
        "RatePlan.Name": "HHC 15k Contact Block - Annual",
        "RatePlanCharge.Id": "2c92a0fc610e738701610ef41f0d2172",
        "RatePlanCharge.BillingPeriod": "Annual",
        "RatePlanCharge.Description": "HHC 15k Contact Block",
        "RatePlanCharge.Quantity": "1",
        "RatePlanCharge.Version": "1",
        "RatePlanCharge.CreatedDate": "1/19/18",
        "RatePlanCharge.EffectiveEndDate": "6/4/18",
        "DefaultPaymentMethod.CreditCardExpirationMonth": "5",
        "DefaultPaymentMethod.CreditCardExpirationYear": "2022",
        "DefaultPaymentMethod.CreditCardMaskNumber": "************8829",
        "ProductRatePlanCharge.Id": "2c92a0fe5b89d5fb015b9e063d321f28",
        "ProductRatePlan.planType__c": "ContactBlocks",
        "ProductRatePlan.planSubType__c": "HHC15k",
        "ProductRatePlan.Id": "2c92a0ff5b89ef4a015b9e05465b4afe",
        "ProductRatePlan.Name": "HHC 15k Contact Block - Annual",
        "Product.productType__c": "HandheldContact",
        "Product.Name": "Handheld Contact",
        "Product.Id": "2c92a0ff5b89ef4b015b9df309627ed3",
        "Product.SKU": "SKU-00000029",
        "Subscription.Id": "2c92a0fc610e738701610ef41ec82153",
        "Subscription.Name": "A-S00092654",
        "Subscription.Status": "Active",
        "Subscription.SubscriptionEndDate": "6/4/18",
        "Subscription.SubscriptionStartDate": "6/4/17",
        "Subscription.TermStartDate": "6/4/17",
        "Subscription.TermEndDate": "6/4/18",
        "Subscription.AutoRenew": "true"
    }].map(rpc => [rpc]);
}
module.exports = {
    getUniqueRatePlans
};


eloqua import create template?


In [ ]:

module.exports = {
    bulkImportTemplate,
    contentCreateTemplate,
    temporaryImportTemplate
}

function bulkImportTemplate(templateId) {
    // replace ID for CDO
    return {
        "name": "Renewals Micro-service - Bulk Import",
        "mapDataCards": "true",
        "mapDataCardsEntityField": "{{Contact.Field(C_EmailAddress)}}",
        "mapDataCardsSourceField": "EmailAddress",
        "mapDataCardsEntityType": "Contact",
        "mapDataCardsCaseSensitiveMatch": "false",
        "updateRule": "always",
        "fields": {
            "AccountId": `{{CustomObject[${templateId}].Field(Account_ID1)}}`,
            "ActProduct": `{{CustomObject[${templateId}].Field(Act_Product1)}}`,
            "EmailAddress": `{{CustomObject[${templateId}].Field(Email_Address1)}}`,
            "Quantity": `{{CustomObject[${templateId}].Field(Quantity1)}}`,
            "Support": `{{CustomObject[${templateId}].Field(Support1)}}`,
            "SupportQuantity": `{{CustomObject[${templateId}].Field(Support_Quantity1)}}`,
            "RenewalDate": `{{CustomObject[${templateId}].Field(Renewal_Date1)}}`,
            "RenewalsStatus": `{{CustomObject[${templateId}].Field(Renewal_Status1)}}`,
            "RepName": `{{CustomObject[${templateId}].Field(Rep_Name1)}}`,
            "RORName": `{{CustomObject[${templateId}].Field(ROR_Name1)}}`,
            "RORNumber": `{{CustomObject[${templateId}].Field(ROR_Number1)}}`,
            "CardExpiration": `{{CustomObject[${templateId}].Field(Card_Expiration1)}}`,
            "State": `{{CustomObject[${templateId}].Field(State1)}}`,
            "Discount": `{{CustomObject[${templateId}].Field(Discount1)}}`,
            "Country": `{{CustomObject[${templateId}].Field(Country1)}}`,
            "Currency": `{{CustomObject[${templateId}].Field(Currency1)}}`
        },
        "identifierFieldName": "EmailAddress"
    }
}

function contentCreateTemplate() {
    return {
        "recordDefinition": {
            "ContactID": "{{Contact.Id}}",
            "EmailAddress": "{{Contact.Field(C_EmailAddress)}}"
        },
        "height": 256,
        "width": 256,
        "editorImageUrl": "https://purchasesprint.actops.com/assets/act-logo-circle.png",
        "requiresConfiguration": false,
    }
}

function temporaryImportTemplate(instance, execution) {
    return {
        "name": "Renewals Micro-service - Bulk Import",
        "mapDataCards": "true",
        "mapDataCardsEntityField": "{{Contact.Field(C_EmailAddress)}}",
        "mapDataCardsSourceField": "EmailAddress",
        "mapDataCardsEntityType": "Contact",
        "mapDataCardsCaseSensitiveMatch": "false",
        "updateRule": "always",
        "fields": {
            "EmailAddress": "{{Contact.Field(C_EmailAddress)}}",
            "Last4DigitsOfCard": "{{Contact.Field(Last_4_Digits_of_Card1)}}",
            "Content": `{{ContentInstance(${instance}).Execution[${execution}]}}`
        },
        "identifierFieldName": "EmailAddress"
    }
}


test eloqua import create template?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var { temporaryImportTemplate } = importer.import('eloqua import create template');

describe('eloqua bulk import definition', () => {
    it('should return the current import definition', () => {
        const dummyInstance = 'instance123';
        const template = temporaryImportTemplate(dummyInstance, 'execution123');
        assert(JSON.stringify(template).includes(dummyInstance));
    })
})


eloqua import blueprints?



In [ ]:

function getImportData() {
    return {
        "AccountId": '1234',
        "ActProduct": 'premium',
        "EmailAddress": "brian.cullinan@swiftpage.com",
        "ExpirationMonth": '12',
        "ExpirationYear": '2017',
        "Last4DigitsOfCard": '1234',
        "Quantity": 5,
        "RenewalDate": '11/12/2017',
        "RenewalsStatus": "Drew loves marketing!",
        "RepName": 'Brian',
        "RORName": 'James',
        "RORNumber": '1111',
        "State": 'AZ',
    }
}

function getOauthToken() {
    return {
        "access_token": "access_token",
        "token_type": "bearer",
        "expires_in": 28800,
        "refresh_token": "refresh_token",
        "expires": (new Date()).getTime() + 28800
    }
}

function getEloquaConfig() {
    return {
        "authorize_uri":"http://localhost:18888/auth/oauth2/authorize",
        "token_uri":"http://localhost:18888/auth/oauth2/token",
        "rest_api_url":"http://localhost:18888",
        "rest_client_id":"client-id",
        "rest_secret":"secret",
        "rest_api_company": "swiftpage",
        "rest_api_user":"username",
        "rest_api_password":"password"
    }
}
module.exports = {
    getImportData,
    getOauthToken,
    getEloquaConfig
};


eloqua existing import?


In [ ]:
var importer = require('../Core');
var request = importer.import('http request polyfill');
var eloquaImport = importer.import('eloqua import service');
var { bulkImportTemplate } = importer.import('eloqua create template');

function getCustomDataObject(eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0/customobjects', // /60/fields',
        method: 'GET',
        headers: eloquaImport.eloquaRequestHeaders(eloquaToken)
    })
        .then(r => r.body.items.filter(i => i.name === 'AUT - NA Renewals')[0])
}

function getImportDefinitions(uri, eloquaToken, eloquaConfig) {
    return request.request({
        followAllRedirects: true,
        uri: eloquaConfig.rest_api_url + '/bulk/2.0' + uri + '/imports',
        method: 'GET',
        headers: eloquaImport.eloquaRequestHeaders(eloquaToken)
    })
        .then(r => r.body.items.filter(i => i.name === bulkImportTemplate(0).name)[0])
}

module.exports = {
    getCustomDataObject,
    getImportDefinitions
};


test eloqua existing import?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var request = importer.import('http request polyfill');
var existing = importer.import('eloqua existing import');
var { bulkImportTemplate } = importer.import('eloqua create template');
var {
    getEloquaConfig,
    getOauthToken,
} = importer.import('eloqua import blueprints');

var eloquaConfig = getEloquaConfig();
var eloquaToken = getOauthToken();
var sandbox = sinon.createSandbox();

describe('eloqua existing import', () => {
        
    afterEach(() => {
        sandbox.restore();
    })

    it('should check if the custom object is configured', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {items: [{name: 'AUT - NA Renewals'}] } }));
        
        return existing.getCustomDataObject({}, eloquaConfig)
            .then(r => {
                assert(r);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
    it('should check if there is already an import definition', () => {
        
        const requestStub = sandbox.stub(request, "request")
            .returns(Promise.resolve({ body: {items: [{name: bulkImportTemplate(0).name, uri: '/imports/1234'}] } }));
        
        return existing.getImportDefinitions('/imports/1234', eloquaToken, eloquaConfig)
            .then(r => {
                assert(r.uri);
                assert(requestStub.calledOnce, 'request should only be called once');
            })
    })
    
});


aws entry point?

In [ ]:
var importer = require('../Core');
var eloquaUpload = importer.import('bulk upload eloqua');
var zuoraExport = importer.import('zuora export month');
var mapper = importer.import('zuora eloqua mapper');

function handler(event, context, callback) {
    const zuoraConfig = {
        "rest_api_user": process.env.ZUORA_API_USER,
        "rest_api_password": process.env.ZUORA_API_PASS,
        "rest_api_url": process.env.ZUORA_API_URL
    };
    const eloquaConfig = {
        "authorize_uri": process.env.ELOQUA_AUTHORIZE_URL,
        "token_uri": process.env.ELOQUA_TOKEN_URL,
        "rest_api_url": process.env.ELOQUA_API_URL,
        "rest_client_id": process.env.ELOQUA_CLIENT_ID,
        "rest_secret": process.env.ELOQUA_CLIENT_SECRET,
        "rest_api_company": process.env.ELOQUA_API_COMPANY,
        "rest_api_user": process.env.ELOQUA_API_USER,
        "rest_api_password": process.env.ELOQUA_API_PASS
    };
    return zuoraExport.getZuoraMonth(0, zuoraConfig)
        .then(records => mapper.mapDataToFields(records))
        .then(accounts => eloquaUpload.bulkUploadEloqua(accounts, eloquaConfig))
        .then(() => callback(null, {
            'statusCode': 200,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Success!'
        }))
        .catch(e => callback(e.message || e.body || e, {
            'statusCode': 500,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Error: ' + e.message
        }))
}
module.exports = {
    handler
};


test aws entry point?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var bundle = importer.import('aws entry point');
var eloquaUpload = importer.import('bulk upload eloqua');
var zuoraExport = importer.import('zuora export month');
var mapper = importer.import('zuora eloqua mapper');

process.env.ZUORA_API_USER = 'devteam@fakepage.com';
process.env.ZUORA_API_USER = 'pass';
process.env.ZUORA_API_USER = 'http://localhost:18888';
var sandbox = sinon.createSandbox();

describe('bulk upload entry point', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should call zuora month export', () => {
        var callback = sinon.spy();
        sandbox.stub(zuoraExport, 'getZuoraMonth').returns(Promise.resolve([]));
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(zuoraExport.getZuoraMonth.calledWithMatch(0));
            });
    });
    
    it('should call data mapper', () => {
        var callback = sinon.spy();
        sandbox.stub(zuoraExport, 'getZuoraMonth').returns(Promise.resolve([]));
        sandbox.stub(mapper, 'mapDataToFields').returns([]);
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(mapper.mapDataToFields.calledWithMatch([]));
            });
    });
    
    it('should call bulk upload', () => {
        var callback = sinon.spy();
        sandbox.stub(zuoraExport, 'getZuoraMonth').returns(Promise.resolve([]));
        sandbox.stub(mapper, 'mapDataToFields').returns([]);
        sandbox.stub(eloquaUpload, 'bulkUploadEloqua').returns([]);
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(eloquaUpload.bulkUploadEloqua.calledWithMatch([]));
            });
    });
})


notify entry point?


In [ ]:
var importer = require('../Core');
var zuoraExport = importer.import('zuora account service');
var eloquaUpload = importer.import('bulk upload eloqua');

function handler(event, context, callback) {
    const zuoraConfig = {
        "rest_api_user": process.env.ZUORA_API_USER,
        "rest_api_password": process.env.ZUORA_API_PASS,
        "rest_api_url": process.env.ZUORA_API_URL
    };
    const eloquaConfig = {
        "authorize_uri": process.env.ELOQUA_AUTHORIZE_URL,
        "token_uri": process.env.ELOQUA_TOKEN_URL,
        "rest_api_url": process.env.ELOQUA_API_URL,
        "rest_client_id": process.env.ELOQUA_CLIENT_ID,
        "rest_secret": process.env.ELOQUA_CLIENT_SECRET,
        "rest_api_company": process.env.ELOQUA_API_COMPANY,
        "rest_api_user": process.env.ELOQUA_API_USER,
        "rest_api_password": process.env.ELOQUA_API_PASS
    };
    var body = event || {};
    try {
        if (event.body || event.queryStringParameters) {
            body = Object.assign(event.body || {}, event.queryStringParameters || {});
        }
    } catch(e) {
        console.log(e);
        callback(e, {
            'statusCode': 500,
            'headers': { 
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*'
            },
            'body': JSON.stringify({'Error': e.message})
        })
        return;
    }
    
    // TODO: add if statement for creating the import template
    // TODO: add an entry point for Zuora subscription callout to update single records in eloqua?
    return zuoraExport.getZuoraAccounts(body, zuoraConfig)
        .then(accounts => eloquaUpload.bulkUploadEloqua(accounts, eloquaConfig, body.instanceId, body.executionId))
        .then(() => callback(null, {
            'statusCode': 200,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Success!'
        }))
        .catch(e => callback(e.message || e.body || e, {
            'statusCode': 500,
            'headers': { 'Content-Type': 'text/plain' },
            'body': 'Error: ' + e.message
        }))
}
module.exports = {
    handler
};


test notify entry point?


In [ ]:
var sinon = require('sinon');
var assert = require('assert');
var importer = require('../Core');
var bundle = importer.import('notify entry point');
var zuoraExport = importer.import('zuora account service');
var eloquaUpload = importer.import('bulk upload eloqua');

var sandbox = sinon.createSandbox();

describe('content notify entry point', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should call zuora export', () => {
        const callback = sinon.spy();
        const dummyBody = {
        }
        
        const requestStub = sandbox.stub(zuoraExport, "getZuoraAccounts")
            .returns(Promise.resolve([]));
        sandbox.stub(eloquaUpload, "bulkUploadEloqua")
            .returns(Promise.resolve(true));
        
        return bundle.handler(dummyBody, null, callback)
            .then(() => {
                assert(callback.calledOnce);
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[0], dummyBody);
            });
    })
    
    it('should call bulk upload', () => {
        const callback = sinon.spy();
        const dummyAccounts = [];
        
        sandbox.stub(zuoraExport, "getZuoraAccounts")
            .returns(Promise.resolve(dummyAccounts));
        const requestStub = sandbox.stub(eloquaUpload, "bulkUploadEloqua")
            .returns(Promise.resolve(true));
        
        return bundle.handler({}, null, callback)
            .then(() => {
                assert(callback.calledOnce);
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[0], dummyAccounts);
            });
    })
})


zuora export month?



In [ ]:
var importer = require('../Core');
var renewalsQuery = importer.import('zuora renewals query');
var zuoraExport = importer.import('zuora to eloqua.ipynb[0]');

function getZuoraMonth(months, zuoraConfig) {
    if(!months) {
        months = 0;
    }
    var start = new Date();
    start.setMonth(start.getMonth() - months);
    start.setDate(1);
    start.setHours(0, 0, 0);
    var end = new Date();
    end.setMonth(end.getMonth() + months + 1);
    end.setDate(1);
    end.setHours(0, 0, 0);
    if(start.getMonth() > end.getMonth()) {
        end.getFullYear(end.getFullYear() + 1)
    }
    
    return zuoraExport.createBulkExportJob(renewalsQuery.getQuery(start.toString(), end.toString()), zuoraConfig)
        .then(exportId => zuoraExport.getBulkExportJobStatus(exportId, zuoraConfig))
        .then(fileId => zuoraExport.getBulkExportFile(fileId, zuoraConfig))
        .then(r => zuoraExport.csvToJson(r))
}
module.exports = {
    getZuoraMonth
};


test zuora export month?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var exporter = importer.import('zuora export month');
var zuoraExport = importer.import('zuora to eloqua.ipynb[0]');
var renewalsQuery = importer.import('zuora renewals query');

var sandbox = sinon.createSandbox();
var zuoraConfig = {
    "rest_api_user":"devteam@fakepage.com",
    "rest_api_password":"pass",
    "rest_api_url": "http://localhost:18888"
};

describe('zuora export month', () => {
    
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should get the query', () => {
        const dummyQuery = 'test query';
        
        sandbox.stub(zuoraExport, "createBulkExportJob").returns(Promise.resolve(''));
        sandbox.stub(zuoraExport, "getBulkExportJobStatus");
        sandbox.stub(zuoraExport, "getBulkExportFile");
        sandbox.stub(zuoraExport, "csvToJson").returns(Promise.resolve([]));
        const queryStub = sandbox.stub(renewalsQuery, "getQuery")
            .returns({ Query: dummyQuery });

        return exporter.getZuoraMonth(0, zuoraConfig).then(result => {
                assert.equal(result.length, 0);
                assert(queryStub.calledOnce, 'getQuery should only be called once');
                const stubCall = queryStub.getCall(0);
                assert.equal(new Date(stubCall.args[0]).getMonth(), (new Date()).getMonth());
        });
    })
    
    it('should call bulk export service', () => {
        const dummyQuery = 'test query';
        
        const exportStub = sandbox.stub(zuoraExport, "createBulkExportJob").returns(Promise.resolve(''));
        sandbox.stub(zuoraExport, "getBulkExportJobStatus");
        sandbox.stub(zuoraExport, "getBulkExportFile");
        sandbox.stub(zuoraExport, "csvToJson").returns(Promise.resolve([]));
        sandbox.stub(renewalsQuery, "getQuery")
            .returns({ Query: dummyQuery });

        return exporter.getZuoraMonth(0, zuoraConfig).then(result => {
                assert.equal(result.length, 0);
                assert(exportStub.calledOnce, 'createBulkExportJob should only be called once');
                const stubCall = exportStub.getCall(0);
                assert.equal(stubCall.args[0].Query, dummyQuery);
        });
    })
    
    it('should convert csv dump to json', () => {
        const dummyCsv = 'this,is,a,test\n1,2,3,4';
        
        sandbox.stub(zuoraExport, "createBulkExportJob").returns(Promise.resolve(''));
        sandbox.stub(zuoraExport, "getBulkExportJobStatus");
        sandbox.stub(zuoraExport, "getBulkExportFile").returns(Promise.resolve(dummyCsv));
        sandbox.stub(renewalsQuery, "getQuery").returns({ Query: 'test query' });
        
        return exporter.getZuoraMonth(0, zuoraConfig).then(result => {
                assert.equal(result[0].test, 4);
        });
    })
})


zuora account service?


In [ ]:
var zuoraExport = importer.import('zuora export service');
var request = importer.import('request polyfill');

module.exports = {
    getZuoraAccounts,
    zuoraQuery
};

function zuoraQuery(query, zuoraConfig) {
    return request.request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/action/query',
        json: {
            queryString: query
        },
        method: 'POST',
        headers: zuoraExport.getAuthHeaders(zuoraConfig)
    });
}

function getContact(email, zuoraConfig) {
    return module.exports.zuoraQuery(`
SELECT AccountId, PersonalEmail, WorkEmail
FROM Contact
WHERE PersonalEmail LIKE '${email}'
OR WorkEmail LIKE '${email}'`, zuoraConfig)
}

function getAccountById(accountId, zuoraConfig) {
    return module.exports.zuoraQuery(`
SELECT Id, Status, Name, Currency, DefaultPaymentMethodId
FROM Account
WHERE Id='${accountId}'`, zuoraConfig);
}

function getPaymentMethod(paymentId, zuoraConfig) {
    return module.exports.zuoraQuery(`
SELECT CreditCardMaskNumber
FROM PaymentMethod
WHERE Id='${paymentId}'`, zuoraConfig);
}

function getAccountLast4Digits(email, zuoraConfig) {
    return getContact(email, zuoraConfig)
        .then(r => getAccountById(r.body.records[0].AccountId, zuoraConfig))
        .then(r => getPaymentMethod(r.body.records[0].DefaultPaymentMethodId, zuoraConfig))
        .then(r => {
            return {
                "EmailAddress": email,
                "Last4DigitsOfCard": r.body.records[0].CreditCardMaskNumber
            }
        })
}

function getZuoraAccounts(notifyRequest, zuoraConfig) {
    if(!notifyRequest.items) {
        return Promise.resolve([]).then(() => {
            throw new Error('Account items not defined, nothing to do!');
        });
    }
    return Promise.all(notifyRequest.items.map(c => {
        return getAccountLast4Digits(c.EmailAddress, zuoraConfig);
    }))
}


test zuora account service?


In [ ]:
var sinon = require('sinon');
var assert = require('assert');
var importer = require('../Core');
var request = importer.import('request polyfill');
var accounts = importer.import('zuora account service');

var sandbox = sinon.createSandbox();
var zuoraConfig = {
    "rest_api_user":"devteam@fakepage.com",
    "rest_api_password":"pass",
    "rest_api_url": "http://localhost:18888"
};

describe('zuora account service', () => {
    afterEach(() => {
        sandbox.restore();
    })
    
    it('should call zuora query', () => {
        const dummyEmail = 'zuora-test@swiftipage.com'
        
        const queryStub = sandbox.stub(request, 'request').returns(Promise.resolve({ body: {records: [{}]} }))
        
        return accounts.getZuoraAccounts({
            items: [{
                 'EmailAddress': dummyEmail
            }]
        }, zuoraConfig).then(r => {
            assert(r);
            const queryCall = queryStub.getCall(0);
            assert(queryCall.args[0].json.queryString.includes(dummyEmail));
        });
    })
    
})


bulk upload eloqua?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var eloquaImport = importer.import('eloqua import service');
var eloquaObjects = importer.import('eloqua existing import');

module.exports = {
    bulkUploadEloqua,
};

function bulkUploadEloqua(accounts, eloquaConfig, instanceId, executionId) {
    var eloquaToken, importUri;
    return eloquaImport.eloquaOauth(eloquaConfig)
        .then(token => {
            eloquaToken = token;
            assert(token.expires > (new Date()).getTime());
            if(instanceId) {
                return eloquaImport.createInstanceDefinition(instanceId, executionId, eloquaToken, eloquaConfig)
            } else {
                // get custom data object ID from API
                var existingId;
                return eloquaObjects.getCustomDataObject(eloquaToken, eloquaConfig)
                    .then(existing => {
                        if(!existing) {
                            throw new Error('Eloqua custom data object not configured.')
                        }
                        existingId = existing.uri.split('/').pop();
                        return eloquaObjects.getImportDefinitions(existing.uri, eloquaToken, eloquaConfig);
                    })
                    .then(importDef => importDef
                          ? importDef.uri
                          : eloquaImport.createImportDefinition(existingId, eloquaToken, eloquaConfig))
            }
        })
        .then(r => {
            importUri = r;
            return eloquaImport.startBulkImportData(accounts, importUri, eloquaToken, eloquaConfig);
        })
        .then(() => eloquaImport.completeBulkImportSync(importUri, eloquaToken, eloquaConfig))
        .then(() => accounts)
}

if(typeof $$ !== 'undefined') {
    $$.async();
    bulkUploadEloqua()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


test bulk upload eloqua?


In [ ]:
var assert = require('assert');
var sinon = require('sinon');
var importer = require('../Core');
var dataImporter = importer.import('bulk upload service');
var eloquaImport = importer.import('eloqua import service');
var eloquaObjects = importer.import('eloqua existing import');
var {
    getEloquaConfig,
    getOauthToken
} = importer.import('eloqua import blueprints');

var eloquaConfig = getEloquaConfig();
var eloquaToken = getOauthToken();
var sandbox = sinon.createSandbox();

describe('eloqua bulk upload', () => {
    
    afterEach(() => {
        sandbox.restore();
    })

    it('should call oauth', () => {
        const requestStub = sandbox.stub(eloquaImport, "eloquaOauth").returns(Promise.resolve(eloquaToken));
        sandbox.stub(eloquaImport, "createImportDefinition");
        sandbox.stub(eloquaImport, "startBulkImportData");
        sandbox.stub(eloquaObjects, "getImportDefinitions");
        sandbox.stub(eloquaObjects, "getCustomDataObject").returns(Promise.resolve({uri: '/customObject/1234'}));
        sandbox.stub(eloquaImport, "completeBulkImportSync").returns(Promise.resolve());
        
        return dataImporter.bulkUploadEloqua({}, eloquaConfig)
            .then(() => {
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[0], eloquaConfig);
            })
    })
    
    it('should call import data', () => {
        sandbox.stub(eloquaImport, "eloquaOauth").returns(Promise.resolve(eloquaToken));
        sandbox.stub(eloquaImport, "createImportDefinition").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        sandbox.stub(eloquaImport, "startBulkImportData").returns(Promise.resolve({ body: {status: 'success'} }));
        sandbox.stub(eloquaObjects, "getImportDefinitions");
        sandbox.stub(eloquaObjects, "getCustomDataObject").returns(Promise.resolve({uri: '/customObject/1234'}));
        const requestStub = sandbox.stub(eloquaImport, "completeBulkImportSync").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        
        return dataImporter.bulkUploadEloqua({}, eloquaConfig)
            .then(() => {
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[1], eloquaToken);
            })
    })
    
    it('should import temporary for each email from Content', () => {
        sandbox.stub(eloquaImport, "eloquaOauth").returns(Promise.resolve(eloquaToken));
        sandbox.stub(eloquaImport, "createInstanceDefinition").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        sandbox.stub(eloquaImport, "startBulkImportData").returns(Promise.resolve({ body: {status: 'success'} }));
        const requestStub = sandbox.stub(eloquaImport, "completeBulkImportSync").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        
        return dataImporter.bulkUploadEloqua({}, eloquaConfig, 'instance123', 'execution123')
            .then(() => {
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[1], eloquaToken);
            })
    })
    
    it('should import an existing definition', () => {
        sandbox.stub(eloquaImport, "eloquaOauth").returns(Promise.resolve(eloquaToken));
        sandbox.stub(eloquaImport, "createImportDefinition").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        sandbox.stub(eloquaImport, "startBulkImportData").returns(Promise.resolve({ body: {status: 'success'} }));
        sandbox.stub(eloquaObjects, "getImportDefinitions").returns(Promise.resolve({uri: '/customObject/1234'}));
        sandbox.stub(eloquaObjects, "getCustomDataObject").returns(Promise.resolve({uri: '/customObject/1234'}));
        const requestStub = sandbox.stub(eloquaImport, "completeBulkImportSync").returns(Promise.resolve({ body: {uri: '/imports/1234'} }));
        
        return dataImporter.bulkUploadEloqua({}, eloquaConfig)
            .then(() => {
                const stubCall = requestStub.getCall(0);
                assert.equal(stubCall.args[1], eloquaToken);
            })
    })
})


sync zuora eloqua end to end?



In [ ]:

describe('zuora to eloqua', () => {
    beforeEach(() => {
    })
    
    it('should export a month of zuora data', () => {
    })
    
    it('should match all products in zuora catalog', () => {
    })
    
    it('should transfer data end-to-end', () => {
    })
    
    /*
    TODO: 
Start a trial through portal
Add / remove support
ACC / distributor updates
Make a purchase
Purchase expires
Change quantity
Cancel subscription
Change quantity within 30 days of renewal
Change quantity within 60 days of renewal
AU/NZ renewals
ACC have eternally free terms
Renew with new subscription, same account
Renew new subscription, same email
*/
    
})


calculate price?


In [ ]:
// given a list of subscription IDs and products, calculate the new subscription total using a catalog export and compare with the preview API

// returns [account number, subtotal, previous discount]
function calculatePrice(subscription, products) {
    const rpcs = _.groupBy(subscription, r => r['RatePlanCharge.Id']);
    const charges = Object.keys(rpcs).map(k => _.sortBy(rpcs[k], r => r['RatePlanCharge.Version']).pop());
    var subtotal = 0;
    var discount;
    var quantity = 0;
    charges
        // TODO: escelate this to mapDataToFields function?
        .filter(c => !(c['ProductRatePlan.Name'] || '').toLowerCase().includes('perpetual'))
        .forEach(charge => {
            const product = products.filter(p => p['id'] === charge['Product.Id'])[0];
            const productRatePlan = product.productRatePlans.filter(p => p['id'] === charge['ProductRatePlan.Id'])[0];
            // select correct price plan for the item
            const productCharge = productRatePlan.productRatePlanCharges.filter(p => p['id'] === charge['ProductRatePlanCharge.Id'])[0];
            const pricing = productCharge.pricing.filter(p => p['currency'] === charge['Account.Currency'])[0];
        
        
            if((charge['ProductRatePlan.Name'] || '').toLowerCase().includes('discount')
                                    || (charge['ProductRatePlan.Name'] || '').toLowerCase().includes('volume')) {
                // TODO: add handler for quantity based discounts?
                discount = pricing.discountPercentage / 100;
            } else {
                const price = pricing.tiers === null
                    ? pricing.price
                    : pricing.tiers.filter(t => charge['RatePlanCharge.Quantity'] >= t['startingUnit']
                                           && charge['RatePlanCharge.Quantity'] <= t['endingUnit'])[0];
                if(typeof price === 'undefined') {
                    throw new Error('unknown rate plan component');
                }
                quantity += parseInt(charge['RatePlanCharge.Quantity']);
                subtotal += parseInt(charge['RatePlanCharge.Quantity']) * price;
            }
        });
    const discounts = charges.filter(c => (c['ProductRatePlan.Name'] || '').toLowerCase().includes('discount') > -1
                                    || (c['ProductRatePlan.Name'] || '').toLowerCase().includes('volume') > -1)
                             .filter(c => !(c['ProductRatePlan.Name'] || '').toLowerCase().includes('diamond')
                                    && !(c['ProductRatePlan.Name'] || '').toLowerCase().includes('distribution'));
    assert(!isNaN(subtotal), 'not a number! ' + JSON.stringify(charges, null, 4))
    //assert(discounts.length <= 1, 'multiple discounts! ' + JSON.stringify(discounts, null, 4))
    return [subscription[0]['Account.AccountNumber'], subtotal, discount, quantity];
}
module.exports = calculatePrice;


calculate price test?


In [ ]:
var _ = require('lodash');
var assert = require('assert');
var xlsx = require('xlsx');
var importer = require('../Core');
var fs = require('fs');

var getZuoraMonth = importer.import('zuora export month');
var calculatePrice = importer.import('calculate price');
var {getCatalog} = importer.import('zuora to eloqua.ipynb[0]');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var zuoraConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/zuoraRest_production.json').toString().trim());

function filterROR(accountROR) {
    return [
            '4001372618',
            '4000919381',
            '411183297',
            '4001358862',
            '4000919006',
            '4000919116',
            '4001372618',
            '411182712',
            '411183101',
            '4001635120',
            '4000919342',
            '4000919068'].filter(ror => (accountROR || '').includes(ror)).length > 0
}

function rorsToAccounts(records) {
    return records.filter(a => a['Account.resellerofRecord__c'])
        .filter(a => filterROR(a['Account.resellerofRecord__c']))
        .map(a => a['Account.AccountNumber']);
}

function totalFilteredRecords(zuoraRecords) {
    const rors = rorsToAccounts(zuoraRecords);
    console.log(rors.length + ' ROR accounts with renewing subscriptions out of ' + zuoraRecords.length + ' records in the dump');
    const recordsToValidate = zuoraRecords
        .filter(s => {
            // TODO: graduate these filters to the main export / import?
            return new Date(s['Subscription.TermEndDate']).getTime() < (new Date('4/1/2018')).getTime()
                && new Date(s['Subscription.TermEndDate']).getTime() >= (new Date('2/27/2018')).getTime()
                && s['Account.Currency'] === 'USD'
                && s['RatePlanCharge.BillingPeriod'] === 'Annual' 
                && !s['ProductRatePlan.Name'].toLowerCase().includes('pro')
        })
    const recordsWithRors = recordsToValidate
        .filter(s => rors.includes(s['Account.AccountNumber']));
    console.log(recordsToValidate.length + ' zuora records to validate');
    console.log(recordsWithRors.length + ' of those are RORs and will also be excluded');
    console.log((recordsToValidate.length - recordsWithRors.length) + ' total');
    return recordsToValidate
        .filter(s => !rors.includes(s['Account.AccountNumber']))
}

function accountTotals(zuoraRecords) {
    const uniqueIds = _.groupBy(zuoraRecords, r => r['Account.Id']);
    console.log(Object.keys(uniqueIds).length + ' unique accounts with expiring subscriptions in February');
    const totals = Object.keys(uniqueIds)
        .map(accountId => {
            return calculatePrice(uniqueIds[accountId], catalog);
        });
    return totals;
}

function verifyMissing(worksheet, totals) {
    
    // TODO: compare with zuora preview API and Jacob's spreadsheet
    const accountIds = totals.map(t => t[0]);
    const worksheetIds = worksheet.map(t => t['Account Number']);
    const missing = worksheet.filter(a => !accountIds.includes(a['Account Number']));
    console.log(missing.length + ' accounts in worksheet, not in zuora export: ');
    //console.log(missing.slice(0, 20));
    const missingZuora = accountIds.filter(a => !worksheetIds.includes(a));
    console.log(missingZuora.length + ' accounts in zuora, missing from worksheet: ');
    console.log(missingZuora.slice(0, 20));
    const verifiableWorksheet = worksheet.filter(a => !missing.includes(a));
    const verifiableTotals = totals.filter(t => worksheetIds.includes(t[0]));
    console.log(verifiableWorksheet.length + ' = ' + verifiableTotals.length + ' verifiable records')
    return {verifiableWorksheet, verifiableTotals};
}

function validateWorksheet(calculatedTotals, zuoraRecords) {
    var workbook = xlsx.readFile(PROFILE_PATH + '/Documents/Marketing_File_Mar_.xlsx');
    var worksheet = xlsx.utils.sheet_to_json(workbook.Sheets['Marketing_File_Mar_']);
    console.log(worksheet.length + ' rows in worksheet');
    const worksheetUSD = worksheet.filter(t => t['Currency'] === 'USD');
    console.log(worksheetUSD.length + ' rows are USD');
    const worksheetFiltered = worksheetUSD.filter(t => filterROR(t['ROR Number'] || '') === false);
    console.log(worksheetFiltered.length + ' rows are USD and not ROR');
    
    const {verifiableWorksheet, verifiableTotals} = verifyMissing(worksheetFiltered, calculatedTotals);
    
    const correct = verifiableWorksheet.filter(a => {
        const realTotal = parseFloat((a['Total 1:'] || '').replace(/[\$,\s]/ig, ''));
        const newTotal = parseFloat((a['Total'] || '').replace(/[\$,\s]/ig, ''));
        const oldTotal = verifiableTotals.filter(t => t[0] === a['Account Number'])[0] || [];
        // TODO: decide what to do with discounts?
        if(a['Account Number'] === 'A00191395') {
            console.log(newTotal);
            console.log(oldTotal);
            console.log(oldTotal[1] - (oldTotal[2] ? (oldTotal[1] * oldTotal[2]) : 0))
        }
        return realTotal === oldTotal[1]
            || newTotal === oldTotal[1]
            || newTotal === oldTotal[1] - (oldTotal[2] ? (oldTotal[1] * oldTotal[2]) : 0)
            || newTotal === oldTotal[1] - (oldTotal[1] * .05);
    });
    const incorrect = verifiableWorksheet.filter(a => !correct.includes(a));
    console.log(incorrect.length + ' incorrect, correct: ' + correct.length + ' out of ' + verifiableTotals.length
                + ' - ' + Math.round(correct.length / verifiableTotals.length * 100) + '%');
    // TODO: fix this
    console.log(incorrect.length + ' + ' + correct.length + ' = ' + (incorrect.length + correct.length));
    // TODO: calculate perpetual price for previousinvoice price comparison?
    // TODO: how to handle australia?
    // TODO: how to handle not in worksheet?
    return {correct, incorrect};
}

var catalog, records;
function compareRecordsCatalog() {
    return (typeof catalog !== 'undefined' ? Promise.resolve(catalog) : getCatalog(zuoraConfig))
        .then(r => (catalog = r))
        .then(() => typeof records !== 'undefined' ? Promise.resolve(records) : getZuoraMonth(6, zuoraConfig))
        .then(r => {
            // filter out the records we aren't validating
            records = r;
            const recordsToValidate = totalFilteredRecords(records);
            const zuoraTotals = accountTotals(recordsToValidate);
            
            const {correct, incorrect} = validateWorksheet(zuoraTotals);
        
            const displayIncorrect = incorrect.map(a => Object.assign(a, {
                incorrect: zuoraTotals.filter(t => t[0] === a['Account Number'])[0] || [],
                subscription: JSON.stringify(recordsToValidate.filter(r => r['Account.AccountNumber'] === a['Account Number']).map(s => s['ProductRatePlan.Name'])),
//                    multiplePrimary:  TODO: check if selecting one product fixes the price
            }))
            const incorrectMultipleSubs = displayIncorrect.filter(a => {
                const subs = recordsToValidate.filter(r => r['Account.AccountNumber'] === a['Account Number']);
                const productGroups = _.groupBy(subs, e => e['ProductRatePlan.Name']);
                return Object.keys(productGroups).length === 0 || Object.keys(productGroups).filter(k => productGroups[k].length > 1).length > 0;
            })
            const verifiableTotal = (correct.length + incorrect.length) - incorrectMultipleSubs.length;
            console.log(incorrectMultipleSubs.length + ' incorrect due to multiple subscriptions, correct minus multiple: '
                        + correct.length + ' out of ' + verifiableTotal + ' - ' + Math.round(correct.length / verifiableTotal * 100) + '%');
            const unaccounted = displayIncorrect.filter(a => !incorrectMultipleSubs.includes(a));
            console.log(unaccounted.length + ' unaccounted for');
            console.log('incorrect sample (' + displayIncorrect.length + '): ');
            console.log(unaccounted); //.map(r => r.subscription)
            
            return zuoraTotals;
        });
}

if(typeof $$ !== 'undefined') {
    $$.async();
    // TODO: pull zuora product catalog
    compareRecordsCatalog()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
    // TODO: this takes to long to download, describe blocks?
}




readme?

# Install

http://www.oracle.com/technetwork/java/javase/downloads/index.html

https://jenkins.io/doc/pipeline/tour/getting-started/

https://nodejs.org

http://docs.aws.amazon.com/cli/latest/userguide/installing.html


Or use Docker

https://github.com/jenkinsci/docker

`docker run -p 8080:8080 -p 50000:50000 -n jenkins jenkins/jenkins:lts`


`docker exec -it -u root jenkins wget -O - https://deb.nodesource.com/setup_8.x | bash && apt-get install -y nodejs zip aws`


# Setup

Create a pipeline and copy the Jenkinsfile, or import from Github

After 61 build tries you should have a screen that looks all green!


# Test

`npm run test`

# Deploy

Create an access key for AWS: https://console.aws.amazon.com/iam/home?nc2=h_m_sc#/security_credential

http://docs.aws.amazon.com/AWSGettingStartedContinuousDeliveryPipeline/latest/GettingStarted/CICD_Jenkins_Pipeline.html

`aws configure`

`aws lambda update-function-code --function-name eloqua_test --zip-file fileb://index.zip`



Get account information

In [ ]:
var _ = require('lodash');
var fs = require('fs');
var importer = require('../Core');
var { request } = importer.import('request polyfill');
var { getAuthHeaders } = importer.import('zuora export service');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var zuoraConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/zuoraRest_production.json').toString().trim());

function zuoraQuery(query) {
    return request({
        followAllRedirects: true,
        uri: zuoraConfig.rest_api_url + '/action/query',
        json: {
            queryString: query
        },
        method: 'POST',
        headers: getAuthHeaders(zuoraConfig)
    });
}

function getContact(email) {
    return zuoraQuery(`SELECT AccountId, PersonalEmail, WorkEmail FROM Contact WHERE PersonalEmail LIKE '${email}' OR WorkEmail LIKE '${email}'`)
}

function getContactByAccount(accountId) {
    return zuoraQuery(`SELECT AccountId, PersonalEmail, WorkEmail FROM Contact WHERE AccountId='${accountId}'`)
}

function getAccountById(accountId) {
    return zuoraQuery(`SELECT Id, Status, Name, Currency, DefaultPaymentMethodId FROM Account WHERE Id='${accountId}'`);
}

function getAccount(accountNumber) {
    return zuoraQuery(`SELECT Id, Status, Name, Currency, DefaultPaymentMethodId FROM Account WHERE AccountNumber='${accountNumber}'`);
}

function getSubscription(accountId) {
    return zuoraQuery(`SELECT Id, Status, TermEndDate FROM Subscription WHERE AccountId='${accountId}'`);
}

function getPaymentMethod(paymentId) {
    return zuoraQuery(`SELECT CreditCardMaskNumber FROM PaymentMethod WHERE Id='${paymentId}'`);
}

function getRatePlans(subscriptionIds) {
    return zuoraQuery(`SELECT Id, Name, SubscriptionId FROM RatePlan WHERE SubscriptionId='${subscriptionIds.join('\', OR SubscriptionId=\'')}'`);
}

if(typeof $$ !== 'undefined') {
$$.async();
    var accountId, paymentId;
    getContact('flyfisher8008@yahoo.com')
        .then(r => {
            console.log(r.body.records);
            return getAccountById(r.body.records[0].AccountId)
        })
        .then(r => {
            console.log(r.body.records);
            accountId = r.body.records[0].Id;
            paymentId = r.body.records[0].DefaultPaymentMethodId
            return getSubscription(accountId)
        })
        .then(r => {
            console.log(r.body.records);
            return getRatePlans(r.body.records.map(r => r.Id));
        })
        .then(r => {
            console.log(_.groupBy(r.body.records, r => r.SubscriptionId))
            return getPaymentMethod(paymentId);
        })
        .then(r => {
            console.log(r.body.records[0].CreditCardMaskNumber)
            return getContactByAccount(accountId);
        })
        .then(r => r.body.records)
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


use the eloqua rest api to retrieve a list of objects


In [ ]:

if(typeof $$ !== 'undefined') {
    $$.async();
    var eloquaToken, eloquaConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/eloqua_production.json').toString().trim());
    eloquaOauth(eloquaConfig)
        .then(r => {
            eloquaToken = r;
            return getEloquaExistingImport(eloquaToken, eloquaConfig)
        })
        /*
        .then(r => {
            // delete import definitions
            const imports = JSON.parse(r.body).items;
            return importer.runAllPromises(imports.map(r => resolve => {
                return request({
                    followAllRedirects: true,
                    uri: eloquaConfig.rest_api_url + '/bulk/2.0' + r.uri,
                    method: 'DELETE',
                    headers: {
                        'Authorization': "Bearer " + eloquaToken.access_token,
                        'Content-Type': 'application/json',
                        'Accept': 'application/json'
                    }
                }).then(r => resolve(r)).catch(e => resolve(e))
            }))
        })
        */
        .then(r => {
            console.log(r);
            $$.mime({'text/html': '<pre>' + JSON.stringify(r, null, 4) + '</pre>'});
        })
        .catch(e => $$.sendError(e))
}

// TODO: find other definitions, compare, and import using the same definition



In [8]:
var _ = require('lodash');
var assert = require('assert');
var xlsx = require('xlsx');
var importer = require('../Core');
var fs = require('fs');

var {getZuoraMonth} = importer.import('zuora export month');
var calculatePrice = importer.import('calculate price');
var {getCatalog} = importer.import('zuora to eloqua.ipynb[0]');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var zuoraConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/zuoraRest_production.json').toString().trim());

var zuora;

if(typeof $$ !== 'undefined') {
    $$.async();
    (zuora ? Promise.resolve(zuora) : getZuoraMonth(6, zuoraConfig))
        .then(r => {
            zuora = r;
        console.log(zuora.length);
            console.log(JSON.stringify(_.uniqBy(zuora, r => r['ProductRatePlan.Name']), null, 4));
        })
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb[19]
reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb[3]
reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb[0]
reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynb[26]
31080
[
    {
        "Account.Id": "2c92a0fb4a38064e014a69432bb9310d",
        "Account.Name": "Soundside Wealth Advisors",
        "Account.AccountNumber": "A00000593",
        "Account.CreatedDate": "12/20/14",
        "Account